In [10]:
#CHECk length
import pandas as pd

# Load your dataset
df = pd.read_csv("try tokens.csv")  # Replace with your file

# Loop through each row of the dataset
for i, row in df.iterrows():
    tokens = row['tokens']  # A string or list of tokens (make sure it's in list format)
    tags = row['tags']  # A string or list of tags (make sure it's in list format)
    polarities = row['polarities']  # A string or list of polarities (make sure it's in list format)
    
    # Convert tokens, tags, and polarities to actual lists if they're in string format
    tokens = tokens.strip('[]').replace("'", "").split(', ')  # Remove brackets and split string to list
    tags = tags.strip('[]').replace("'", "").split(', ')  # Remove brackets and split string to list
    polarities = polarities.strip('[]').replace("'", "").split(', ')  # Remove brackets and split string to list
    
    # Check if the lengths match
    if len(tokens) != len(tags) or len(tags) != len(polarities):
        print(f"Mismatch at index {i}:")
        print(f"Tokens: {tokens} (Length: {len(tokens)})")
        print(f"Tags: {tags} (Length: {len(tags)})")
        print(f"Polarities: {polarities} (Length: {len(polarities)})")
    else:
        print(f"Row {i} has matching lengths.")


Row 0 has matching lengths.
Row 1 has matching lengths.
Row 2 has matching lengths.
Row 3 has matching lengths.
Row 4 has matching lengths.
Row 5 has matching lengths.
Row 6 has matching lengths.
Row 7 has matching lengths.
Row 8 has matching lengths.
Row 9 has matching lengths.
Row 10 has matching lengths.
Row 11 has matching lengths.
Row 12 has matching lengths.
Row 13 has matching lengths.
Row 14 has matching lengths.
Row 15 has matching lengths.
Row 16 has matching lengths.
Row 17 has matching lengths.
Row 18 has matching lengths.
Row 19 has matching lengths.
Row 20 has matching lengths.
Row 21 has matching lengths.
Row 22 has matching lengths.
Row 23 has matching lengths.
Row 24 has matching lengths.
Row 25 has matching lengths.
Row 26 has matching lengths.
Row 27 has matching lengths.
Row 28 has matching lengths.
Row 29 has matching lengths.
Row 30 has matching lengths.
Row 31 has matching lengths.
Row 32 has matching lengths.
Row 33 has matching lengths.
Row 34 has matching leng

In [3]:
#define custom datasets
'''import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer

class CustomDataset(Dataset):
    def __init__(self, dataframe, tokenizer):
        self.data = dataframe
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        tokens = self.data.iloc[idx]['tokens']
        tags = self.data.iloc[idx]['tags']
        polarities = self.data.iloc[idx]['polarities']

        # Tokenize the input text
        encoding = self.tokenizer(tokens, is_split_into_words=True, return_tensors='pt', padding='max_length', truncation=True, max_length=128)
        
        # Convert tags and polarities to tensor
        tags = torch.tensor(tags, dtype=torch.long)
        polarities = torch.tensor(polarities, dtype=torch.long)

        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'tags': tags,
            'polarities': polarities
        }
        '''
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer

class CustomDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=128):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length  # Set the maximum length for padding

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Extract tokens, tags, and polarities for a specific row
        tokens = self.data.iloc[idx]['tokens']
        tags = self.data.iloc[idx]['tags']
        polarities = self.data.iloc[idx]['polarities']

        # Tokenize the input text (tokens should be a list of words)
        encoding = self.tokenizer(tokens, is_split_into_words=True, return_tensors='pt', padding='max_length', truncation=True, max_length=self.max_length)
        
        # Convert the tags and polarities to integers (they might be in string format like '[-1, -1, 0, -1]')
        # Convert the string to a list and then to tensor
        tags = list(map(int, tags.strip('][').split(', ')))  # Convert string to list of integers
        polarities = list(map(int, polarities.strip('][').split(', ')))  # Same for polarities
        
        # Pad tags and polarities if needed
        if len(tags) < self.max_length:
            tags = tags + [0] * (self.max_length - len(tags))  # Pad with 0 if shorter
        if len(polarities) < self.max_length:
            polarities = polarities + [0] * (self.max_length - len(polarities))  # Pad with 0 if shorter
        
        # Convert tags and polarities to tensors
        tags = torch.tensor(tags, dtype=torch.long)
        polarities = torch.tensor(polarities, dtype=torch.long)

        return {
            'input_ids': encoding['input_ids'].squeeze(),  # Remove extra dimension
            'attention_mask': encoding['attention_mask'].squeeze(),  # Remove extra dimension
            'tags': tags,
            'polarities': polarities
        }





In [11]:
#This code splits your dataset
from sklearn.model_selection import train_test_split

# Load your data
df = pd.read_csv("try tokens.csv")

# Split data into training and temporary set (which will be split into validation and test sets)
train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42)

# Split the temporary set into validation and test sets
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# Save the splits to CSV files
train_df.to_csv("mrt_train.csv", index=False)
val_df.to_csv("mrt_val.csv", index=False)
test_df.to_csv("mrt_test.csv", index=False)
print(df.columns)



Index(['tokens', 'tags', 'polarities'], dtype='object')


In [ ]:
#shuffle
import pandas as pd

# Load your dataset
df = pd.read_csv("try tokens.csv")

# Shuffle the rows
df_shuffled = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Save the shuffled dataset to a new CSV file
df_shuffled.to_csv("try tokens.csv", index=False)

In [26]:
#model definition 
from transformers import BertModel, BertPreTrainedModel, BertConfig
import torch
from torch import nn

class bert_ATE(BertPreTrainedModel):
    def __init__(self, config):
        super(bert_ATE, self).__init__(config)
        self.bert = BertModel(config)
        self.classifier = nn.Linear(config.hidden_size, 3)  # Maps hidden size to 3 output classes
        self.loss_fn = nn.CrossEntropyLoss()

    def forward(self, input_ids, attention_mask=None, labels=None):
        bert_outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_state = bert_outputs.last_hidden_state  # Shape: [batch_size, seq_len, hidden_size]
        logits = self.classifier(last_hidden_state)  # Shape: [batch_size, seq_len, 3]

        if labels is not None:
            loss = self.loss_fn(logits.view(-1, 3), labels.view(-1))  # Flatten both tensors
            return {"loss": loss, "logits": logits}
        else:
            return {"logits": logits}

class bert_ABSA(BertPreTrainedModel):
    def __init__(self, config):
        super(bert_ABSA, self).__init__(config)
        self.bert = BertModel(config)
        self.dropout = nn.Dropout(p=0.3)
        self.classifier = nn.Linear(config.hidden_size, 3)  # 3 classes
        self.loss_fn = nn.CrossEntropyLoss()

    def forward(self, input_ids, attention_mask=None, token_type_ids=None, labels=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        
        sequence_output = outputs.last_hidden_state  # ✅ MUST use token-level output
        sequence_output = self.dropout(sequence_output)
        logits = self.classifier(sequence_output)  # Shape: (batch_size, sequence_length, 3)

        if labels is not None:
            # flatten batch and sequence dims for CrossEntropyLoss
            loss = self.loss_fn(logits.view(-1, 3), labels.view(-1))
            return {"loss": loss, "logits": logits}
        else:
            return {"logits": logits}




In [27]:
#training
from torch.optim import AdamW
from tqdm import tqdm

def train(model, dataloader, optimizer, device):
    model.train()
    total_loss = 0
    for batch in tqdm(dataloader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['polarities'].to(device)
        labels = labels + 1  # shift -1 → 0, 0 → 1, 1 → 2


        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask, labels=labels)
        loss = outputs['loss']
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    return total_loss / len(dataloader)


In [28]:
#evaluation 

def evaluate(model, dataloader, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in tqdm(dataloader):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['polarities'].to(device)
            labels = labels + 1  # Use 'tags' for ATE and 'polarities' for ABSA

            outputs = model(input_ids, attention_mask, labels=labels)
            loss = outputs['loss']

            total_loss += loss.item()
    return total_loss / len(dataloader)


In [29]:
#load data and initialize dataloaders

# Load your data
train_df = pd.read_csv("restaurants_train.csv")
val_df = pd.read_csv("restaurants_val.csv")
test_df = pd.read_csv("restaurants_test.csv")

# Initialize the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Create dataset objects
train_dataset = CustomDataset(train_df, tokenizer)
val_dataset = CustomDataset(val_df, tokenizer)
test_dataset = CustomDataset(test_df, tokenizer)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)


In [ ]:
#train and evaluate model 
import torch

# Initialize model, optimizer, and device
model_name = 'bert-base-uncased'
config = BertConfig.from_pretrained(model_name)
model = bert_ABSA(config)  # Use bert_ATE for ATE task
optimizer = AdamW(model.parameters(), lr=2e-5)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Training loop
num_epochs = 3
for epoch in range(num_epochs):
    train_loss = train(model, train_loader, optimizer, device)
    val_loss = evaluate(model, val_loader, device)
    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss}, Validation Loss: {val_loss}')

# Testing
test_loss = evaluate(model, test_loader, device)
print(f'Test Loss: {test_loss}')


  0%|                                                                                          | 0/211 [00:00<?, ?it/s]

In [14]:
import pandas as pd
import ast

# Load your CSV
df = pd.read_csv("try tokens.csv")  # replace with your actual file name

# Convert stringified lists to real Python lists and build final structure
final_data = []
for i, row in df.iterrows():
    tokens = ast.literal_eval(row['tokens'])
    tags = ast.literal_eval(row['tags'])
    polarities = ast.literal_eval(row['polarities'])
    final_data.append([tokens, tags, polarities])

# Preview a few entries
# Preview all entries
for item in final_data:
    print(f"{item},")




[['the', 'station', 'is', 'clean'], [0, 0, 0, 1], [0, 0, 0, 1]],
[['staff', 'were', 'not', 'helpful'], [0, 0, 0, 1], [-1, 0, -1, 1]],
[['easy', 'access', 'to', 'bus'], [1, 2, 0, 1], [1, 1, 0, 1]],
[['the', 'seats', 'were', 'uncomfortable'], [0, 1, 0, 1], [0, -1, 0, -1]],
[['good', 'facilities', 'like', 'vending'], [1, 1, 0, 1], [1, 1, 0, 1]],
[['it', 'was', 'a', 'normal', 'ride'], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]],
[['dirty', 'platform', 'with', 'rubbish'], [1, 2, 0, 0], [-1, -1, 0, -1]],
[['the', 'escalator', 'was', 'broken'], [0, 1, 0, 1], [0, -1, 0, -1]],
[['friendly', 'staff', 'at', 'the', 'counter'], [1, 1, 0, 0, 1], [1, 1, 0, 0, 1]],
[['access', 'was', 'difficult'], [1, 0, 1], [-1, 0, -1]],
[['cleanliness', 'was', 'good'], [1, 0, 1], [1, 0, 1]],
[['the', 'aircon', 'was', 'not', 'working'], [0, 1, 0, 0, 0], [0, -1, 0, -1, -1]],
[['felt', 'unsafe', 'at', 'the', 'station'], [0, 1, 0, 0, 0], [0, -1, 0, 0, 0]],
[['easy', 'to', 'connect', 'to', 'taxi'], [1, 0, 1, 0, 1], [1, 0, 1, 0, 1]

In [16]:
import csv

data =[
[['the', 'station', 'is', 'clean'], [0, 0, 0, 1], [0, 0, 0, 1]],
[['staff', 'were', 'not', 'helpful'], [0, 0, 0, 1], [-1, 0, -1, 1]],
[['easy', 'access', 'to', 'bus'], [1, 2, 0, 1], [1, 1, 0, 1]],
[['the', 'seats', 'were', 'uncomfortable'], [0, 1, 0, 1], [0, -1, 0, -1]],
[['good', 'facilities', 'like', 'vending'], [1, 1, 0, 1], [1, 1, 0, 1]],
[['it', 'was', 'a', 'normal', 'ride'], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]],
[['dirty', 'platform', 'with', 'rubbish'], [1, 2, 0, 0], [-1, -1, 0, -1]],
[['the', 'escalator', 'was', 'broken'], [0, 1, 0, 1], [0, -1, 0, -1]],
[['friendly', 'staff', 'at', 'the', 'counter'], [1, 1, 0, 0, 1], [1, 1, 0, 0, 1]],
[['access', 'was', 'difficult'], [1, 0, 1], [-1, 0, -1]],
[['cleanliness', 'was', 'good'], [1, 0, 1], [1, 0, 1]],
[['the', 'aircon', 'was', 'not', 'working'], [0, 1, 0, 0, 0], [0, -1, 0, -1, -1]],
[['felt', 'unsafe', 'at', 'the', 'station'], [0, 1, 0, 0, 0], [0, -1, 0, 0, 0]],
[['easy', 'to', 'connect', 'to', 'taxi'], [1, 0, 1, 0, 1], [1, 0, 1, 0, 1]],
[['lacking', 'proper', 'signage'], [1, 1, 1], [-1, -1, -1]],
[['the', 'service', 'was', 'slow'], [0, 1, 0, 0], [0, -1, 0, -1]],
[['comfortable', 'seats', 'on', 'the', 'train'], [1, 1, 0, 0, 0], [1, 1, 0, 0, 0]],
[['dirty', 'smell', 'in', 'the', 'station'], [1, 1, 0, 0, 0], [-1, -1, 0, 0, -1]],
[['felt', 'secure', 'with', 'security', 'cameras'], [0, 1, 0, 1, 1], [0, 1, 0, 1, 1]],
[['good', 'access', 'for', 'wheelchair'], [1, 1, 0, 1], [1, 1, 0, 1]],
[['the', 'toilets', 'were', 'not', 'clean'], [0, 1, 0, 0, 0], [0, -1, 0, -1, -1]],
[['staff', 'provided', 'good', 'assistance'], [1, 0, 1, 1], [1, 0, 1, 1]],
[['crowded', 'and', 'uncomfortable'], [1, 0, 1], [-1, 0, -1]],
[['easy', 'walk', 'to', 'nearby', 'bus', 'stop'], [1, 1, 0, 0, 1, 2], [1, 1, 0, 0, 0, 0]],
[['vending', 'machine', 'was', 'out', 'of', 'order'], [1, 2, 0, 0, 0, 0], [-1, -1, 0, -1, -1, -1]],
[['service', 'improvement', 'needed'], [1, 1, 0], [-1, -1, 0]],
[['spacious', 'and', 'comfortable', 'ride'], [1, 0, 1, 0], [1, 1, 1, 1]],
[['bad', 'smell', 'near', 'the', 'toilet'], [1, 1, 0, 0, 1], [-1, -1, 0, 0, -1]],
[['felt', 'safe', 'with', 'guards', 'around'], [0, 1, 0, 1, 1], [0, 1, 0, 1, 1]],
[['a', 'very', 'spacious', 'station'], [0, 0, 1, 0], [0, 0, 1, 0]],
[['lack', 'of', 'parking', 'nearby'], [0, 0, 1, 0], [0, 0, -1, 0]],
[['the', 'signage', 'was', 'confusing'], [0, 1, 0, 0], [0, -1, 0, -1]],
[['poor', 'service', 'overall'], [1, 1, 0], [-1, -1, 0]],
[['comfortable', 'temperature'], [1, 1], [1, 1]],
[['litter', 'on', 'the', 'platform'], [1, 0, 0, 1], [-1, 0, 0, -1]],
[['felt', 'watched', 'by', 'cameras'], [0, 0, 0, 1], [0, 0, 0, 0]],
[['easy', 'to', 'find', 'bus', 'connections'], [1, 0, 0, 1, 2], [1, 0, 0, 0, 0]],
[['vending', 'machines', 'not', 'working'], [1, 2, 0, 0], [-1, -1, -1, -1]],
[['service', 'was', 'acceptable'], [1, 0, 0], [0, 0, 0]],
[['spacious', 'waiting', 'area', 'with', 'seats'], [1, 1, 1, 0, 1], [1, 1, 1, 0, 1]],
[['bad', 'odor', 'in', 'the', 'lift'], [1, 1, 0, 0, 1], [-1, -1, 0, 0, -1]],
[['guards', 'were', 'visible', 'and', 'helpful'], [1, 0, 1, 0, 1], [1, 0, 1, 0, 1]],
[['limited', 'access', 'for', 'strollers'], [1, 0, 1, 0], [-1, 0, -1, 0]],
[['the', 'lift', 'was', 'out', 'of', 'service'], [0, 1, 0, 0, 0, 0], [0, -1, 0, -1, -1, -1]],
[['helpful', 'staff', 'at', 'the', 'information', 'desk'], [1, 1, 0, 0, 1, 2], [1, 1, 0, 0, 1, 1]],
[['very', 'crowded', 'during', 'peak', 'hours'], [0, 1, 0, 0, 0], [0, -1, 0, 0, 0]],
[['direct', 'connection', 'to', 'shopping', 'mall'], [1, 1, 0, 0, 0], [1, 1, 0, 0, 0]],
[['old', 'and', 'poorly', 'maintained'], [0, 0, 0, 0], [0, 0, -1, -1]],
[['the', 'announcement', 'system', 'was', 'bad'], [0, 1, 2, 0, 0], [0, -1, -1, 0, -1]],
[['uncomfortable', 'plastic', 'seats'], [1, 1, 1], [-1, -1, -1]],
[['some', 'dirty', 'patches', 'on', 'the', 'floor'], [0, 1, 1, 0, 0, 1], [0, -1, -1, 0, 0, -1]],
[['felt', 'a', 'bit', 'unsafe', 'at', 'night'], [0, 0, 0, 0, 0, 0], [0, 0, 0, -1, 0, -1]],
[['easy', 'access', 'via', 'the', 'new', 'bridge'], [1, 1, 0, 0, 0, 0], [1, 1, 0, 0, 0, 0]],
[['the', 'parking', 'was', 'expensive'], [0, 1, 0, 0], [0, -1, 0, -1]],
[['slow', 'service', 'at', 'the', 'ticket', 'machines'], [1, 1, 0, 0, 1, 2], [-1, -1, 0, 0, -1, -1]],
[['a', 'clean', 'and', 'bright', 'station'], [0, 1, 0, 1, 0], [0, 1, 1, 1, 0]],
[['lack', 'of', 'seats', 'on', 'the', 'platform'], [0, 0, 1, 0, 0, 1], [0, 0, -1, 0, 0, -1]],
[['the', 'security', 'felt', 'adequate'], [0, 1, 0, 1], [0, 1, 0, 1]],
[['good', 'connectivity', 'with', 'other', 'public', 'transport'], [1, 1, 0, 0, 0, 0], [1, 1, 0, 0, 0, 0]],
[['vending', 'machine', 'accepted', 'only', 'cash'], [1, 2, 0, 0, 0], [0, 0, 0, 0, 0]],
[['efficient', 'and', 'friendly', 'service'], [1, 0, 1, 1], [1, 1, 1, 1]],
[['comfortable', 'and', 'quiet', 'ride'], [1, 0, 1, 1], [1, 1, 1, 1]],
[['felt', 'ignored', 'by', 'the', 'staff'], [0, 0, 0, 0, 1], [0, -1, -1, 0, -1]],
[['easy', 'access', 'for', 'bikes'], [1, 1, 0, 1], [1, 1, 0, 0]],
[['limited', 'seating', 'available'], [1, 1, 1], [-1, -1, -1]],
[['the', 'information', 'was', 'outdated'], [0, 1, 0, 0], [0, -1, 0, -1]],
[['poor', 'lighting', 'on', 'the', 'platform'], [1, 1, 0, 0, 1], [-1, -1, 0, 0, -1]],
[['the', 'MRT', 'is', 'very', 'clean'], [0, 0, 0, 0, 1], [1, 1, 0, 0, 1]],
[['seats', 'are', 'comfortable'], [1, 0, 1], [1, 0, 1]],
[['felt', 'safe', 'on', 'the', 'train'], [0, 1, 0, 0, 0], [1, 1, 0, 0, 0]],
[['good', 'service', 'from', 'staff'], [1, 1, 0, 0], [1, 1, 0, 0]],
[['clean', 'and', 'modern', 'facilities'], [1, 0, 1, 1], [1, 0, 1, 1]],
[['easy', 'access', 'to', 'platforms'], [1, 1, 0, 0], [1, 1, 0, 0]],
[['trains', 'are', 'always', 'clean'], [1, 0, 0, 1], [1, 0, 0, 1]],
[['comfortable', 'and', 'spacious', 'seats'], [1, 0, 1, 1], [1, 0, 1, 1]],
[['felt', 'safe', 'and', 'secure'], [1, 0, 1, 1], [1, 0, 1, 1]],
[['friendly', 'and', 'efficient', 'service'], [1, 0, 1, 1], [1, 0, 1, 1]],
[['cleanliness', 'is', 'well', 'maintained'], [1, 0, 0, 1], [1, 0, 0, 1]],
[['seats', 'are', 'very', 'comfortable'], [1, 0, 0, 1], [1, 0, 0, 1]],
[['unguarded', 'and', 'unprotected', 'areas'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['antiquated', 'and', 'archaic', 'equipment'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['lax', 'and', 'delayed', 'response_time'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['unhygienic', 'and', 'foul-smelling', 'bathrooms'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['stiff', 'and', 'uncomfortable', 'seating'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['shaky', 'and', 'rough', 'commute'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['overcrowded', 'and', 'congested', 'spaces'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['unavailable', 'and', 'non-existent', 'amenities'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['terrible', 'and', 'abysmal', 'service_quality'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['felt', 'safe', 'traveling', 'at', 'night'], [1, 0, 0, 0, 0], [1, 0, 0, 0, 0]],
[['excellent', 'customer', 'service'], [1, 1, 0], [1, 1, 0]],
[['modern', 'and', 'clean', 'facilities'], [1, 0, 1, 1], [1, 0, 1, 1]],
[['good', 'access', 'for', 'disabled'], [1, 1, 0, 0], [1, 1, 0, 0]],
[['trains', 'are', 'spotlessly', 'clean'], [1, 0, 0, 1], [1, 0, 0, 1]],
[['comfortable', 'and', 'quiet', 'ride'], [1, 0, 1, 1], [1, 0, 1, 1]],
[['very', 'safe', 'and', 'reliable'], [1, 0, 1, 1], [1, 0, 1, 1]],
[['fast', 'and', 'efficient', 'service'], [1, 0, 1, 1], [1, 0, 1, 1]],
[['cleanliness', 'is', 'top', 'notch'], [1, 0, 0, 1], [1, 0, 0, 1]],
[['comfortable', 'seats', 'and', 'ample', 'legroom'], [1, 1, 0, 1, 1], [1, 1, 0, 1, 1]],
[['felt', 'very', 'safe', 'and', 'secure'], [1, 0, 0, 1, 1], [1, 0, 0, 1, 1]],
[['friendly', 'and', 'helpful', 'staff'], [1, 0, 1, 1], [1, 0, 1, 1]],
[['modern', 'facilities', 'and', 'good', 'maintenance'], [1, 1, 0, 1, 1], [1, 1, 0, 1, 1]],
[['easy', 'access', 'to', 'all', 'areas'], [1, 1, 0, 0, 0], [1, 1, 0, 0, 0]],
[['trains', 'are', 'always', 'clean', 'and', 'tidy'], [1, 0, 0, 1, 0, 1], [1, 0, 0, 1, 0, 1]],
[['comfortable', 'and', 'smooth', 'ride'], [1, 0, 1, 1], [1, 0, 1, 1]],
[['safe', 'and', 'reliable', 'transportation'], [1, 0, 1, 1], [1, 0, 1, 1]],
[['difficult', 'access', 'for', 'elderly'], [1, 0, 1, 0], [-1, 0, -1, 0]],
[['the', 'lifts', 'were', 'slow'], [0, 1, 0, 0], [0, -1, 0, -1]],
[['rude', 'staff', 'at', 'information'], [1, 1, 0, 1], [-1, -1, 0, -1]],
[['easy', 'connection', 'to', 'other', 'lines'], [1, 1, 0, 0, 0], [1, 1, 0, 0, 0]],
[['not', 'well-maintained', 'facilities'], [0, 0, 1], [0, -1, -1]],
[['the', 'announcements', 'were', 'clear'], [0, 1, 0, 1], [0, 1, 0, 1]],
[['uncomfortable', 'waiting', 'area'], [1, 1, 1], [-1, -1, -1]],
[['dirty', 'seats', 'on', 'the', 'train'], [1, 1, 0, 0, 0], [-1, -1, 0, 0, -1]],
[['felt', 'a', 'sense', 'of', 'security'], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1]],
[['good', 'access', 'from', 'the', 'main', 'road'], [1, 1, 0, 0, 0, 0], [1, 1, 0, 0, 0, 0]],
[['the', 'ticket', 'counter', 'was', 'efficient'], [0, 1, 2, 0, 1], [0, 0, 0, 0, 1]],
[['unfriendly', 'staff', 'at', 'the', 'gate'], [1, 1, 0, 0, 1], [-1, -1, 0, 0, -1]],
[['efficient', 'and', 'courteous', 'service'], [1, 0, 1, 1], [1, 0, 1, 1]],
[['high', 'level', 'of', 'cleanliness'], [1, 1, 0, 0], [1, 1, 0, 0]],
[['comfortable', 'and', 'well-maintained', 'seats'], [1, 0, 1, 1], [1, 0, 1, 1]],
[['felt', 'safe', 'and', 'protected'], [1, 0, 1, 1], [1, 0, 1, 1]],
[['excellent', 'and', 'fast', 'service'], [1, 0, 1, 1], [1, 0, 1, 1]],
[['modern', 'and', 'well-equipped', 'facilities'], [1, 0, 1, 1], [1, 0, 1, 1]],
[['easy', 'and', 'convenient', 'access'], [1, 0, 1, 1], [1, 0, 1, 1]],
[['trains', 'are', 'consistently', 'clean'], [1, 0, 0, 1], [1, 0, 0, 1]],
[['comfortable', 'and', 'enjoyable', 'ride'], [1, 0, 1, 1], [1, 0, 1, 1]],
[['safe', 'and', 'secure', 'journey'], [1, 0, 1, 1], [1, 0, 1, 1]],
[['efficient', 'and', 'professional', 'service'], [1, 0, 1, 1], [1, 0, 1, 1]],
[['superb', 'cleanliness', 'standards'], [1, 1, 0], [1, 1, 0]],
[['very', 'comfortable', 'seats'], [1, 0, 1], [1, 0, 1]],
[['felt', 'extremely', 'safe'], [1, 0, 1], [1, 0, 1]],
[['outstanding', 'customer', 'service'], [1, 1, 0], [1, 1, 0]],
[['highly', 'accessible', 'stations'], [1, 1, 0], [1, 1, 0]],
[['trains', 'are', 'always', 'immaculate'], [1, 0, 0, 1], [1, 0, 0, 1]],
[['comfortable', 'and', 'relaxing', 'ride'], [1, 0, 1, 1], [1, 0, 1, 1]],
[['safe', 'and', 'reliable', 'travel'], [1, 0, 1, 1], [1, 0, 1, 1]],
[['efficient', 'and', 'timely', 'service'], [1, 0, 1, 1], [1, 0, 1, 1]],
[['the', 'mrt', 'is', 'not', 'clean'], [0, 0, 0, 0, 1], [-1, -1, 0, 0, -1]],
[['seats', 'are', 'uncomfortable'], [1, 0, 1], [-1, 0, -1]],
[['felt', 'unsafe', 'on', 'the', 'train'], [0, 1, 0, 0, 0], [-1, -1, 0, 0, 0]],
[['poor', 'service', 'from', 'staff'], [1, 1, 0, 0], [-1, -1, 0, 0]],
[['dirty', 'and', 'old', 'facilities'], [1, 0, 1, 1], [-1, 0, -1, -1]],
[['lack', 'of', 'cleanliness'], [1, 1, 0], [-1, -1, 0]],
[['very', 'uncomfortable', 'seats'], [1, 0, 1], [-1, 0, -1]],
[['felt', 'unsafe', 'traveling', 'at', 'night'], [1, 0, 0, 0, 0], [-1, 0, 0, 0, 0]],
[['terrible', 'customer', 'service'], [1, 1, 0], [-1, -1, 0]],
[['old', 'and', 'dirty', 'facilities'], [1, 0, 1, 1], [-1, 0, -1, -1]],
[['poor', 'access', 'for', 'disabled'], [1, 1, 0, 0], [-1, -1, 0, 0]],

[['inaccessible', 'stations', 'for', 'many'], [1, 1, 0, 0], [-1, -1, 0, 0]],
[['trains', 'are', 'consistently', 'filthy'], [1, 0, 0, 1], [-1, 0, 0, -1]],
[['uncomfortable', 'and', 'rough', 'ride'], [1, 0, 1, 1], [-1, 0, -1, -1]],
[['unsafe', 'and', 'unreliable', 'transport'], [1, 0, 1, 1], [-1, 0, -1, -1]],
[['inefficient', 'and', 'rude', 'service'], [1, 0, 1, 1], [-1, 0, -1, -1]],
[['cleanliness', 'could', 'be', 'better'], [1, 0, 0, 0], [0, 0, 0, 0]],
[['seats', 'are', 'okay'], [1, 0, 0], [0, 0, 0]],
[['felt', 'reasonably', 'safe'], [1, 0, 0], [0, 0, 0]],
[['average', 'customer', 'service'], [1, 1, 0], [0, 0, 0]],
[['facilities', 'are', 'adequate'], [1, 0, 0], [0, 0, 0]],
[['access', 'is', 'passable'], [1, 0, 0], [0, 0, 0]],
[['trains', 'are', 'sometimes', 'clean'], [1, 0, 0, 0], [0, 0, 0, 0]],
[['ride', 'was', 'neither', 'comfortable', 'nor', 'uncomfortable'], [1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]],
[['safety', 'level', 'seems', 'average'], [1, 0, 0, 0], [0, 0, 0, 0]],
[['service', 'was', 'acceptable'], [1, 0, 0], [0, 0, 0]],
[['cleanliness', 'is', 'average'], [1, 0, 0], [0, 0, 0]],
[['seats', 'are', 'moderately', 'comfortable'], [1, 0, 0, 0], [0, 0, 0, 0]],
[['felt', 'generally', 'safe'], [1, 0, 0], [0, 0, 0]],
[['standard', 'customer', 'service'], [1, 1, 0], [0, 0, 0]],
[['facilities', 'are', 'decent'], [1, 0, 0], [0, 0, 0]],
[['access', 'is', 'fair'], [1, 0, 0], [0, 0, 0]],
[['trains', 'are', 'usually', 'clean'], [1, 0, 0, 0], [0, 0, 0, 0]],
[['ride', 'was', 'reasonably', 'comfortable'], [1, 0, 0, 0], [0, 0, 0, 0]],
[['safety', 'is', 'satisfactory'], [1, 0, 0], [0, 0, 0]],
[['service', 'is', 'adequate'], [1, 0, 0], [0, 0, 0]],
[['the', 'MRT', 'is', 'very', 'clean'], [0, 0, 0, 0, 1], [1, 1, 0, 0, 1]],
[['seats', 'are', 'comfortable', 'and', 'plush'], [1, 0, 1, 0, 1], [1, 0, 1, 0, 1]],
    [['trains', 'are', 'almost', 'never', 'clean'], [0, 0, 0, 0, 1], [-1, 0, 0, -1, -1]],
[['uncomfortable', 'and', 'bumpy', 'ride'], [1, 0, 1, 1], [-1, 0, -1, -1]],
[['very', 'unsafe', 'and', 'unreliable'], [1, 0, 1, 1], [-1, 0, -1, -1]],
[['slow', 'and', 'unhelpful', 'service'], [1, 0, 1, 1], [-1, 0, -1, -1]],
[['extremely', 'dirty', 'conditions'], [1, 1, 0], [-1, -1, 0]],
[['highly', 'uncomfortable', 'seats'], [1, 1, 1], [-1, -1, -1]],
[['felt', 'extremely', 'unsafe'], [1, 0, 1], [-1, 0, -1]],
[['worst', 'customer', 'service', 'ever'], [1, 1, 0, 0], [-1, -1, 0, 0]],
[['dilapidated', 'and', 'filthy', 'facilities'], [1, 0, 1, 1], [-1, 0, -1, -1]],
[['felt', 'safe', 'with', 'security', 'present'], [0, 1, 0, 1, 0], [1, 1, 0, 1, 0]],
[['good', 'customer', 'service', 'overall'], [1, 1, 0, 0], [1, 1, 0, 0]],
[['modern', 'and', 'well-maintained', 'facilities'], [1, 0, 1, 1], [1, 0, 1, 1]],
[['easy', 'access', 'for', 'all', 'passengers'], [1, 1, 0, 0, 0], [1, 1, 0, 0, 0]],
[['trains', 'are', 'always', 'clean', 'and', 'tidy'], [1, 0, 0, 1, 0, 1], [1, 0, 0, 1, 0, 1]],
[['comfortable', 'temperature', 'and', 'smooth', 'ride'], [1, 1, 0, 1, 1], [1, 1, 0, 1, 1]],
[['very', 'safe', 'and', 'secure', 'environment'], [1, 0, 1, 1, 0], [1, 0, 1, 1, 0]],
[['friendly', 'and', 'efficient', 'service', 'provided'], [1, 0, 1, 1, 0], [1, 0, 1, 1, 0]],
[['cleanliness', 'is', 'a', 'top', 'priority'], [1, 0, 0, 1, 0], [1, 0, 0, 1, 0]],
[['seats', 'are', 'very', 'comfortable', 'and', 'spacious'], [1, 0, 0, 1, 0, 1], [1, 0, 0, 1, 0, 1]],
[['felt', 'safe', 'traveling', 'alone', 'at', 'night'], [1, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0]],
[['excellent', 'and', 'prompt', 'service'], [1, 0, 1, 1], [1, 0, 1, 1]],
[['difficult', 'access', 'to', 'platforms'], [1, 1, 0, 0], [-1, -1, 0, 0]],
[['trains', 'are', 'rarely', 'clean'], [1, 0, 0, 1], [-1, 0, 0, -1]],
[['uncomfortable', 'and', 'noisy', 'ride'], [1, 0, 1, 1], [-1, 0, -1, -1]],
[['felt', 'unsafe', 'and', 'insecure'], [1, 0, 1, 1], [-1, 0, -1, -1]],
[['slow', 'and', 'inefficient', 'service'], [1, 0, 1, 1], [-1, 0, -1, -1]],
[['modern', 'and', 'clean', 'facilities', 'available'], [1, 0, 1, 1, 0], [1, 0, 1, 1, 0]],
[['good', 'access', 'for', 'families', 'with', 'strollers'], [1, 1, 0, 0, 0, 0], [1, 1, 0, 0, 0, 0]],
[['trains', 'are', 'spotlessly', 'clean', 'and', 'well-maintained'], [1, 0, 0, 1, 0, 1], [1, 0, 0, 1, 0, 1]],
[['comfortable', 'and', 'quiet', 'journey'], [1, 0, 1, 1], [1, 0, 1, 1]],
[['very', 'safe', 'and', 'reliable', 'transport'], [1, 0, 1, 1, 0], [1, 0, 1, 1, 0]],
[['fast', 'and', 'efficient', 'service', 'always'], [1, 0, 1, 1, 0], [1, 0, 1, 1, 0]],
[['cleanliness', 'is', 'consistently', 'high'], [1, 0, 0, 1], [1, 0, 0, 1]],
[['comfortable', 'seats', 'with', 'good', 'back', 'support'], [1, 1, 0, 1, 0, 0], [1, 1, 0, 1, 0, 0]],
[['felt', 'very', 'safe', 'and', 'secure', 'throughout'], [1, 0, 0, 1, 1, 0], [1, 0, 0, 1, 1, 0]],
[['friendly', 'and', 'helpful', 'staff', 'everywhere'], [1, 0, 1, 1, 0], [1, 0, 1, 1, 0]],
[['modern', 'facilities', 'and', 'excellent', 'maintenance'], [1, 1, 0, 1, 0], [1, 1, 0, 1, 0]],
[['the', 'mrt', 'is', 'not', 'very', 'clean'], [0, 0, 0, 0, 0, 1], [-1, -1, 0, 0, 0, -1]],
[['seats', 'are', 'uncomfortable', 'and', 'worn', 'out'], [1, 0, 1, 0, 1, 0], [-1, 0, -1, 0, -1, 0]],
[['felt', 'unsafe', 'on', 'the', 'train', 'at', 'night'], [0, 1, 0, 0, 0, 0, 0], [-1, -1, 0, 0, 0, 0, 0]],
[['poor', 'customer', 'service', 'experience'], [1, 1, 0, 0], [-1, -1, 0, 0]],
[['dirty', 'and', 'old', 'facilities', 'in', 'poor', 'condition'], [1, 0, 1, 1, 0, 0, 0], [-1, 0, -1, -1, 0, 0, 0]],
[['difficult', 'access', 'to', 'platforms', 'for', 'elderly'], [1, 1, 0, 0, 0, 0], [-1, -1, 0, 0, 0, 0]],
[['trains', 'are', 'rarely', 'clean', 'and', 'well-maintained'], [1, 0, 0, 1, 0, 1], [-1, 0, 0, -1, 0, -1]],
[['uncomfortable', 'and', 'noisy', 'commute'], [1, 0, 1, 1], [-1, 0, -1, -1]],
[['felt', 'unsafe', 'and', 'insecure', 'throughout', 'the', 'journey'], [1, 0, 1, 1, 0, 0, 0], [-1, 0, -1, -1, 0, 0, 0]],
[['slow', 'and', 'inefficient', 'service', 'delivery', 'always'], [1, 0, 1, 1, 0, 0], [-1, 0, -1, -1, 0, 0]],
[['lack', 'of', 'cleanliness', 'is', 'a', 'major', 'issue'], [1, 1, 0, 0, 0, 0, 0], [-1, -1, 0, 0, 0, 0, 0]],
[['easy', 'access', 'to', 'all', 'train', 'lines'], [1, 1, 0, 0, 0, 0], [1, 1, 0, 0, 0, 0]],
[['trains', 'are', 'always', 'clean', 'and', 'well-organized'], [1, 0, 0, 1, 0, 1], [1, 0, 0, 1, 0, 1]],
[['comfortable', 'and', 'smooth', 'commute'], [1, 0, 1, 1], [1, 0, 1, 1]],
[['safe', 'and', 'reliable', 'public', 'transportation'], [1, 0, 1, 1, 0], [1, 0, 1, 1, 0]],
[['efficient', 'and', 'courteous', 'service', 'provided', 'by', 'staff'], [1, 0, 1, 1, 0, 0, 0], [1, 0, 1, 1, 0, 0, 0]],
[['high', 'level', 'of', 'cleanliness', 'observed'], [1, 1, 0, 0, 0], [1, 1, 0, 0, 0]],
[['comfortable', 'and', 'well-maintained', 'seats', 'always'], [1, 0, 1, 1, 0], [1, 0, 1, 1, 0]],
[['felt', 'safe', 'and', 'protected', 'throughout', 'the', 'journey'], [1, 0, 1, 1, 0, 0, 0], [1, 0, 1, 1, 0, 0, 0]],
[['excellent', 'and', 'fast', 'service', 'delivery'], [1, 0, 1, 1, 0], [1, 0, 1, 1, 0]],
[['modern', 'and', 'well-equipped', 'facilities', 'available', 'for', 'everyone'], [1, 0, 1, 1, 0, 0, 0], [1, 0, 1, 1, 0, 0, 0]],
[['easy', 'and', 'convenient', 'access', 'to', 'all', 'areas', 'of', 'the', 'station'], [1, 0, 1, 1, 0, 0, 0, 0, 0, 0], [1, 0, 1, 1, 0, 0, 0, 0, 0, 0]],
[['dirty', 'and', 'stained', 'seats'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['stuffy', 'and', 'poorly-ventilated', 'trains'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['faulty', 'and', 'defective', 'machinery'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['run-down', 'and', 'obsolete', 'facilities'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['dismissive', 'and', 'unfriendly', 'staff'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['grimy', 'and', 'littered', 'platforms'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['narrow', 'and', 'packed', 'carriages'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['unguarded', 'and', 'unprotected', 'areas'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['archaic', 'and', 'antiquated', 'systems'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['lax', 'and', 'delayed', 'response'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['unhygienic', 'and', 'foul-smelling', 'toilets'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['stiff', 'and', 'uncomfortable', 'seating'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['shaky', 'and', 'rough', 'commute'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['overcrowded', 'and', 'congested', 'spaces'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['unavailable', 'and', 'non-existent', 'services'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['abysmal', 'and', 'terrible', 'customer_service'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['trains', 'are', 'consistently', 'clean', 'and', 'well-maintained'], [1, 0, 0, 1, 0, 1], [1, 0, 0, 1, 0, 1]],
[['comfortable', 'and', 'enjoyable', 'commute'], [1, 0, 1, 1], [1, 0, 1, 1]],
[['safe', 'and', 'secure', 'travel', 'experience'], [1, 0, 1, 1, 0], [1, 0, 1, 1, 0]],
[['efficient', 'and', 'professional', 'service', 'provided'], [1, 0, 1, 1, 0], [1, 0, 1, 1, 0]],
[['superb', 'cleanliness', 'standards', 'maintained'], [1, 1, 0, 0], [1, 1, 0, 0]],
[['very', 'comfortable', 'and', 'plush', 'seats'], [1, 0, 1, 0, 1], [1, 0, 1, 0, 1]],
[['felt', 'extremely', 'safe', 'and', 'secure', 'always'], [1, 0, 1, 0, 1, 0], [1, 0, 1, 0, 1, 0]],
[['outstanding', 'customer', 'service', 'experience'], [1, 1, 0, 0], [1, 1, 0, 0]],
[['highly', 'accessible', 'stations', 'for', 'everyone'], [1, 1, 0, 0, 0], [1, 1, 0, 0, 0]],
[['trains', 'are', 'always', 'immaculate', 'and', 'well-kept'], [1, 0, 0, 1, 0, 1], [1, 0, 0, 1, 0, 1]],
[['comfortable', 'and', 'relaxing', 'journey', 'always'], [1, 0, 1, 1, 0], [1, 0, 1, 1, 0]],
[['safe', 'and', 'reliable', 'public', 'transportation', 'system'], [1, 0, 1, 1, 0, 0], [1, 0, 1, 1, 0, 0]],
[['efficient', 'and', 'timely', 'service', 'delivery', 'always'], [1, 0, 1, 1, 0, 0], [1, 0, 1, 1, 0, 0]],
[['very', 'uncomfortable', 'and', 'hard', 'seats'], [1, 0, 1, 0, 1], [-1, 0, -1, 0, -1]],
[['terrible', 'customer', 'service', 'provided', 'by', 'staff'], [1, 1, 0, 0, 0, 0], [-1, -1, 0, 0, 0, 0]],
[['old', 'and', 'dirty', 'facilities', 'in', 'a', 'state', 'of', 'disrepair'], [1, 0, 1, 1, 0, 0, 0, 0, 0], [-1, 0, -1, -1, 0, 0, 0, 0, 0]],
[['poor', 'access', 'for', 'disabled', 'passengers'], [1, 1, 0, 0, 0], [-1, -1, 0, 0, 0]],
[['trains', 'are', 'almost', 'never', 'clean', 'and', 'well-maintained'], [1, 0, 0, 0, 1, 0, 1], [-1, 0, 0, -1, -1, 0, -1]],
[['uncomfortable', 'and', 'bumpy', 'commute', 'experience'], [1, 0, 1, 1, 0], [-1, 0, -1, -1, 0]],
[['very', 'unsafe', 'and', 'unreliable', 'public', 'transport'], [1, 0, 1, 1, 0, 0], [-1, 0, -1, -1, 0, 0]],
[['slow', 'and', 'unhelpful', 'service', 'from', 'the', 'personnel'], [1, 0, 1, 1, 0, 0, 0], [-1, 0, -1, -1, 0, 0, 0]],
[['extremely', 'dirty', 'and', 'unhygienic', 'conditions'], [1, 1, 0, 0, 0], [-1, -1, 0, 0, 0]],
[['highly', 'uncomfortable', 'and', 'hard', 'seats', 'always'], [1, 1, 1, 0, 1, 0], [-1, -1, -1, 0, -1, 0]],
[['felt', 'extremely', 'unsafe', 'and', 'vulnerable'], [1, 0, 1, 0, 1], [-1, 0, -1, 0, -1]],
[['worst', 'customer', 'service', 'ever', 'experienced'], [1, 1, 0, 0, 0], [-1, -1, 0, 0, 0]],
[['dilapidated', 'and', 'filthy', 'facilities', 'in', 'a', 'terrible', 'state'], [1, 0, 1, 1, 0, 0, 0, 0], [-1, 0, -1, -1, 0, 0, 0, 0]],
[['inaccessible', 'stations', 'for', 'many', 'passengers'], [1, 1, 0, 0, 0], [-1, -1, 0, 0, 0]],
[['trains', 'are', 'consistently', 'filthy', 'and', 'poorly', 'maintained'], [1, 0, 0, 1, 0, 0, 1], [-1, 0, 0, -1, -1, 0, -1]],
[['uncomfortable', 'and', 'rough', 'commute', 'experience', 'always'], [1, 0, 1, 1, 0, 0], [-1, 0, -1, -1, 0, 0]],
[['inefficient', 'and', 'rude', 'service', 'from', 'the', 'staff', 'always'], [1, 0, 1, 1, 0, 0, 0, 0], [-1, 0, -1, -1, 0, 0, 0, 0]],
[['cleanliness', 'could', 'be', 'significantly', 'improved'], [1, 0, 0, 0, 0], [0, 0, 0, 0, 0]],
[['seats', 'are', 'just', 'okay', 'nothing', 'special'], [1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]],
[['felt', 'reasonably', 'safe', 'during', 'the', 'day'], [1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]],
[['average', 'customer', 'service', 'experience', 'overall'], [1, 1, 0, 0, 0], [0, 0, 0, 0, 0]],
[['facilities', 'are', 'adequate', 'for', 'basic', 'needs'], [1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]],
[['access', 'is', 'passable', 'for', 'most', 'people'], [1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]],
[['trains', 'are', 'sometimes', 'clean', 'but', 'not', 'always'], [1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]],
[['ride', 'was', 'neither', 'comfortable', 'nor', 'uncomfortable', 'just', 'neutral'], [1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0]],
[['safety', 'level', 'seems', 'average', 'during', 'peak', 'hours'], [1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]],
[['service', 'was', 'acceptable', 'but', 'could', 'be', 'better'], [1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]],
[['unguarded', 'and', 'unprotected', 'areas'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['antiquated', 'and', 'archaic', 'technology'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['lax', 'and', 'delayed', 'response'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['unhygienic', 'and', 'foul-smelling', 'restrooms'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['stiff', 'and', 'uncomfortable', 'seating'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['shaky', 'and', 'rough', 'ride'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['congested', 'and', 'overcrowded', 'spaces'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['non-existent', 'and', 'unavailable', 'services'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['abysmal', 'and', 'terrible', 'customer_service'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['stained', 'and', 'dirty', 'upholstery'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['poorly-ventilated', 'and', 'stuffy', 'carriages'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['faulty', 'and', 'defective', 'systems'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['run-down', 'and', 'obsolete', 'infrastructure'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['dismissive', 'and', 'unfriendly', 'staff_attitude'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['grimy', 'and', 'littered', 'platforms'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['narrow', 'and', 'packed', 'corridors'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['cleanliness', 'is', 'average', 'and', 'could', 'use', 'improvement'], [1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]],
[['seats', 'are', 'moderately', 'comfortable', 'for', 'short', 'trips'], [1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]],
[['felt', 'generally', 'safe', 'but', 'would', 'prefer', 'more', 'security'], [1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0]],
[['standard', 'customer', 'service', 'with', 'no', 'major', 'issues'], [1, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]],
[['facilities', 'are', 'decent', 'but', 'could', 'be', 'more', 'modern'], [1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0]],
[['access', 'is', 'fair', 'but', 'could', 'be', 'improved', 'for', 'disabled'], [1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0]],
[['trains', 'are', 'usually', 'clean', 'but', 'can', 'get', 'dirty', 'sometimes'], [1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0]],
[['ride', 'was', 'reasonably', 'comfortable', 'for', 'the', 'duration'], [1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]],
[['safety', 'is', 'satisfactory', 'overall', 'with', 'room', 'for', 'improvement'], [1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0]],
[['service', 'is', 'adequate', 'but', 'staff', 'could', 'be', 'more', 'helpful'], [1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0]],
[['the', 'MRT', 'is', 'very', 'clean', 'and', 'well-maintained'], [0, 0, 0, 0, 1, 2, 1], [1, 1, 0, 0, 1, 1, 1]],
[['seats', 'are', 'comfortable', 'and', 'spacious'], [1, 0, 1, 2, 1], [1, 0, 1, 1, 1]],
[['felt', 'safe', 'and', 'secure', 'on', 'the', 'train'], [0, 1, 2, 1, 0, 0, 0], [1, 1, 1, 1, 0, 0, 0]],
[['good', 'customer', 'service', 'is', 'provided'], [1, 1, 0, 2, 1], [1, 1, 0, 1, 1]],
[['modern', 'and', 'clean', 'facilities', 'are', 'available'], [1, 0, 1, 1, 2, 1], [1, 0, 1, 1, 1, 1]],
[['easy', 'access', 'to', 'platforms', 'is', 'a', 'plus'], [1, 1, 0, 0, 2, 0, 1], [1, 1, 0, 0, 1, 0, 1]],
[['trains', 'are', 'always', 'clean', 'and', 'tidy', 'and', 'well-lit'], [1, 0, 0, 1, 2, 1, 2, 1], [1, 0, 0, 1, 1, 1, 1, 1]],
[['comfortable', 'temperature', 'and', 'smooth', 'ride', 'is', 'great'], [1, 1, 0, 1, 1, 2, 1], [1, 1, 0, 1, 1, 1, 1]],
[['very', 'safe', 'and', 'reliable', 'environment', 'is', 'ensured'], [1, 0, 1, 1, 0, 2, 1], [1, 0, 1, 1, 0, 1, 1]],
[['friendly', 'and', 'efficient', 'service', 'is', 'provided', 'always'], [1, 0, 1, 1, 2, 1, 0], [1, 0, 1, 1, 1, 1, 0]],
[['the', 'mrt', 'is', 'not', 'very', 'clean', 'and', 'poorly', 'maintained'], [0, 0, 0, 0, 0, 1, 2, 0, 1], [-1, -1, 0, 0, 0, -1, -1, 0, -1]],
[['seats', 'are', 'uncomfortable', 'and', 'worn', 'out', 'and', 'dirty'], [1, 0, 1, 0, 1, 0, 2, 1], [-1, 0, -1, 0, -1, 0, -1, -1]],
[['felt', 'unsafe', 'on', 'the', 'train', 'at', 'night', 'and', 'insecure'], [0, 1, 0, 0, 0, 0, 0, 2, 0], [-1, -1, 0, 0, 0, 0, 0, -1, -1]],
[['poor', 'customer', 'service', 'experience', 'is', 'always', 'delivered'], [0, 1, 1, 0, 0, 0, 0], [-1, -1, 0, 0, -1, 0, -1]],
[['dirty', 'and', 'old', 'facilities', 'are', 'in', 'poor', 'condition', 'and', 'poorly', 'maintained'], [1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1], [-1, 0, -1, -1, -1, 0, 0, 0, -1, 0, -1]],
[['difficult', 'access', 'to', 'platforms', 'for', 'elderly', 'is', 'a', 'major', 'issue'], [1, 1, 0, 0, 0, 0, 0, 0, 0, 0], [-1, -1, 0, 0, 0, 0, -1, 0, 0, -1]],
[['trains', 'are', 'rarely', 'clean', 'and', 'poorly', 'maintained', 'and', 'often', 'late'], [1, 0, 0, 1, 0, 0, 1, 0, 0, 1], [-1, 0, 0, -1, -1, 0, -1, -1, 0, -1]],
[['cleanliness', 'is', 'well', 'maintained', 'and', 'evident'], [1, 0, 0, 1, 2, 1], [1, 0, 0, 1, 1, 1]],
[['seats', 'are', 'very', 'comfortable', 'and', 'spacious', 'and', 'clean'], [1, 0, 0, 1, 2, 1, 2, 1], [1, 0, 0, 1, 1, 1, 1, 1]],
[['felt', 'safe', 'traveling', 'alone', 'at', 'night', 'and', 'secure'], [1, 0, 0, 0, 0, 0, 2, 1], [1, 0, 0, 0, 0, 0, 1, 1]],
[['excellent', 'and', 'prompt', 'service', 'is', 'delivered'], [1, 0, 1, 1, 2, 1], [1, 0, 1, 1, 1, 1]],
[['modern', 'and', 'clean', 'facilities', 'are', 'readily', 'available'], [1, 0, 1, 1, 2, 0, 1], [1, 0, 1, 1, 1, 0, 1]],
[['good', 'access', 'for', 'families', 'with', 'strollers', 'is', 'important'], [1, 1, 0, 0, 0, 0, 2, 1], [1, 1, 0, 0, 0, 0, 1, 1]],
[['trains', 'are', 'spotlessly', 'clean', 'and', 'well-maintained', 'and', 'comfortable'], [1, 0, 0, 1, 2, 1, 2, 1], [1, 0, 0, 1, 1, 1, 1, 1]],
[['comfortable', 'and', 'quiet', 'journey', 'is', 'appreciated'], [1, 0, 1, 1, 2, 1], [1, 0, 1, 1, 1, 1]],
[['fast', 'and', 'efficient', 'service', 'is', 'always', 'provided'], [1, 0, 1, 1, 2, 0, 1], [1, 0, 1, 1, 1, 0, 1]],
[['cleanliness', 'is', 'consistently', 'high', 'and', 'noticeable'], [1, 0, 0, 1, 2, 1], [1, 0, 0, 1, 1, 1]],
[['comfortable', 'seats', 'with', 'good', 'back', 'support', 'are', 'provided'], [1, 1, 0, 1, 0, 0, 2, 1], [1, 1, 0, 1, 0, 0, 1, 1]],
[['felt', 'very', 'safe', 'and', 'secure', 'throughout', 'the', 'journey', 'and', 'comfortable'], [1, 0, 0, 1, 1, 0, 0, 0, 2, 1], [1, 0, 0, 1, 1, 0, 0, 0, 1, 1]],
[['grimy', 'and', 'littered', 'stations'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['narrow', 'and', 'packed', 'aisles'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['unguarded', 'and', 'unprotected', 'areas'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['antiquated', 'and', 'archaic', 'technology'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['lax', 'and', 'delayed', 'response'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['unhygienic', 'and', 'foul-smelling', 'restrooms'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['stiff', 'and', 'uncomfortable', 'seating'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['shaky', 'and', 'rough', 'ride'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['congested', 'and', 'overcrowded', 'spaces'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['non-existent', 'and', 'unavailable', 'services'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['abysmal', 'and', 'terrible', 'customer_service'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['stained', 'and', 'unclean', 'surfaces'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['stuffy', 'and', 'poorly-ventilated', 'coaches'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['faulty', 'and', 'defective', 'equipment'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['run-down', 'and', 'obsolete', 'systems'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['dismissive', 'and', 'unfriendly', 'attitude'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['grimy', 'and', 'littered', 'stations'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['narrow', 'and', 'packed', 'aisles'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['friendly', 'and', 'helpful', 'staff', 'are', 'present', 'everywhere'], [1, 0, 1, 1, 2, 1, 0], [1, 0, 1, 1, 1, 1, 0]],
[['modern', 'facilities', 'and', 'excellent', 'maintenance', 'are', 'provided', 'regularly'], [1, 1, 0, 1, 0, 2, 1, 0], [1, 1, 0, 1, 0, 1, 0, 0]],
[['easy', 'access', 'to', 'all', 'train', 'lines', 'is', 'guaranteed'], [1, 1, 0, 0, 0, 0, 2, 1], [1, 1, 0, 0, 0, 0, 1, 1]],
[['trains', 'are', 'always', 'clean', 'and', 'well-organized', 'and', 'on-time'], [1, 0, 0, 1, 2, 1, 2, 1], [1, 0, 0, 1, 1, 1, 1, 1]],
[['comfortable', 'and', 'smooth', 'commute', 'is', 'always', 'guaranteed'], [1, 0, 1, 1, 2, 0, 1], [1, 0, 1, 1, 1, 0, 1]],
[['efficient', 'and', 'courteous', 'service', 'provided', 'by', 'staff', 'is', 'appreciated'], [1, 0, 1, 1, 0, 0, 0, 2, 1], [1, 0, 1, 1, 0, 0, 0, 1, 1]],
[['high', 'level', 'of', 'cleanliness', 'is', 'always', 'observed'], [1, 1, 0, 0, 2, 0, 1], [1, 1, 0, 0, 1, 0, 1]],
[['comfortable', 'and', 'well-maintained', 'seats', 'are', 'always', 'available'], [1, 0, 1, 1, 2, 0, 1], [1, 0, 1, 1, 1, 0, 1]],
[['felt', 'safe', 'and', 'protected', 'throughout', 'the', 'journey', 'and', 'secure'], [1, 0, 1, 1, 0, 0, 0, 2, 1], [1, 0, 1, 1, 0, 0, 0, 1, 1]],
[['excellent', 'and', 'fast', 'service', 'delivery', 'is', 'guaranteed'], [1, 0, 1, 1, 0, 2, 1], [1, 0, 1, 1, 0, 1, 1]],
[['modern', 'and', 'well-equipped', 'facilities', 'are', 'available', 'for', 'everyone', 'and', 'well-maintained'], [1, 0, 1, 1, 2, 1, 0, 0, 2, 1], [1, 0, 1, 1, 1, 1, 0, 0, 1, 1]],
[['easy', 'and', 'convenient', 'access', 'to', 'all', 'areas', 'of', 'the', 'station', 'is', 'provided'], [1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 2, 1], [1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1]],
[['trains', 'are', 'consistently', 'clean', 'and', 'well-maintained', 'and', 'on-time'], [1, 0, 0, 1, 2, 1, 2, 1], [1, 0, 0, 1, 1, 1, 1, 1]],
[['comfortable', 'and', 'enjoyable', 'commute', 'is', 'always', 'guaranteed', 'and', 'smooth'], [1, 0, 1, 1, 2, 0, 1, 2, 1], [1, 0, 1, 1, 1, 0, 1, 1, 1]],
[['safe', 'and', 'secure', 'travel', 'experience', 'is', 'always', 'prioritized'], [1, 0, 1, 1, 0, 2, 0, 1], [1, 0, 1, 1, 0, 1, 0, 1]],
[['efficient', 'and', 'professional', 'service', 'is', 'provided', 'always', 'and', 'courteously'], [1, 0, 1, 1, 2, 1, 0, 2, 1], [1, 0, 1, 1, 1, 1, 0, 1, 1]],
[['superb', 'cleanliness', 'standards', 'are', 'maintained', 'always'], [1, 1, 0, 2, 1, 0], [1, 1, 0, 1, 1, 0]],
[['very', 'comfortable', 'and', 'plush', 'seats', 'are', 'provided'], [1, 0, 1, 0, 1, 2, 1], [1, 0, 1, 0, 1, 1, 1]],
[['felt', 'extremely', 'safe', 'and', 'secure', 'always', 'and', 'comfortable'], [1, 0, 1, 0, 1, 0, 2, 1], [1, 0, 1, 0, 1, 0, 1, 1]],
[['outstanding', 'customer', 'service', 'experience', 'is', 'always', 'delivered'], [1, 1, 0, 0, 2, 0, 1], [1, 1, 0, 0, 1, 0, 1]],
[['highly', 'accessible', 'stations', 'are', 'available', 'for', 'everyone', 'and', 'well-equipped'], [1, 1, 0, 2, 1, 0, 0, 2, 1], [1, 1, 0, 1, 1, 0, 0, 1, 1]],
[['trains', 'are', 'always', 'immaculate', 'and', 'well-kept', 'and', 'on-time'], [1, 0, 0, 1, 2, 1, 2, 1], [1, 0, 0, 1, 1, 1, 1, 1]],
[['comfortable', 'and', 'relaxing', 'journey', 'is', 'always', 'guaranteed', 'and', 'smooth'], [1, 0, 1, 1, 2, 0, 1, 2, 1], [1, 0, 1, 1, 1, 0, 1, 1, 1]],
[['safe', 'and', 'reliable', 'public', 'transportation', 'system', 'is', 'always', 'prioritized'], [1, 0, 1, 1, 0, 0, 2, 0, 1], [1, 0, 1, 1, 0, 0, 1, 0, 1]],
[['efficient', 'and', 'timely', 'service', 'delivery', 'is', 'always', 'provided', 'and', 'appreciated'], [1, 0, 1, 1, 0, 2, 0, 1, 2, 1], [1, 0, 1, 1, 0, 1, 0, 1, 1, 1]],
[['uncomfortable', 'and', 'noisy', 'commute', 'is', 'always', 'experienced'], [1, 0, 1, 1, 2, 0, 0], [-1, 0, -1, -1, -1, 0, -1]],
[['felt', 'unsafe', 'and', 'insecure', 'throughout', 'the', 'journey', 'and', 'vulnerable'], [1, 0, 1, 1, 0, 0, 0, 2, 0], [-1, 0, -1, -1, 0, 0, 0, -1, -1]],
[['slow', 'and', 'inefficient', 'service', 'delivery', 'is', 'always', 'provided', 'and', 'unhelpful'], [1, 0, 1, 1, 0, 2, 0, 0, 2, 0], [-1, 0, -1, -1, 0, -1, 0, -1, -1, -1]],
[['lack', 'of', 'cleanliness', 'is', 'a', 'major', 'issue', 'and', 'very', 'noticeable'], [1, 1, 0, 2, 0, 0, 0, 2, 0, 0], [-1, -1, 0, -1, 0, 0, -1, -1, 0, -1]],
[['very', 'uncomfortable', 'and', 'hard', 'seats', 'are', 'always', 'provided'], [1, 0, 1, 0, 1, 2, 0, 0], [-1, 0, -1, 0, -1, -1, 0, -1]],
[['felt', 'unsafe', 'traveling', 'alone', 'at', 'night', 'frequently', 'and', 'vulnerable'], [1, 0, 0, 0, 0, 0, 0, 2, 0], [-1, 0, 0, 0, 0, 0, 0, -1, -1]],
[['terrible', 'customer', 'service', 'provided', 'by', 'staff', 'is', 'always', 'experienced'], [1, 1, 0, 0, 0, 0, 2, 0, 0], [-1, -1, 0, 0, 0, 0, -1, 0, -1]],
[['old', 'and', 'dirty', 'facilities', 'are', 'in', 'a', 'state', 'of', 'disrepair', 'and', 'poorly', 'maintained'], [1, 0, 1, 1, 2, 0, 0, 0, 0, 0, 2, 0, 0], [-1, 0, -1, -1, -1, 0, 0, 0, 0, 0, -1, 0, -1]],
[['trains', 'are', 'almost', 'never', 'clean', 'and', 'poorly', 'maintained', 'and', 'frequently', 'late'], [1, 0, 0, 0, 0, 2, 0, 0, 2, 0, 0], [-1, 0, 0, -1, -1, -1, 0, -1, -1, 0, -1]],
[['uncomfortable', 'and', 'bumpy', 'commute', 'experience', 'is', 'always', 'terrible'], [1, 0, 1, 1, 0, 2, 0, 0], [-1, 0, -1, -1, 0, -1, 0, -1]],
[['very', 'unsafe', 'and', 'unreliable', 'public', 'transport', 'is', 'always', 'experienced', 'and', 'insecure'], [1, 0, 1, 1, 0, 0, 2, 0, 0, 2, 0], [-1, 0, -1, -1, 0, 0, -1, 0, -1, -1, -1]],
[['slow', 'and', 'unhelpful', 'service', 'from', 'the', 'personnel', 'is', 'always', 'provided', 'and', 'rude'], [1, 0, 1, 1, 0, 0, 0, 2, 0, 0, 2, 0], [-1, 0, -1, -1, 0, 0, 0, -1, 0, -1, -1, -1]],
[['extremely', 'dirty', 'and', 'unhygienic', 'conditions', 'are', 'always', 'present', 'and', 'very', 'noticeable'], [1, 1, 0, 0, 0, 2, 0, 0, 2, 0, 0], [-1, -1, 0, 0, 0, -1, 0, -1, -1, 0, -1]],
[['highly', 'uncomfortable', 'and', 'hard', 'seats', 'are', 'always', 'provided', 'and', 'worn', 'out'], [1, 1, 1, 0, 1, 2, 0, 0, 2, 0, 0], [-1, -1, -1, 0, -1, -1, 0, -1, -1, 0, -1]],
[['felt', 'extremely', 'unsafe', 'and', 'vulnerable', 'is', 'a', 'constant', 'experience'], [1, 0, 1, 0, 1, 2, 0, 0, 0], [-1, 0, -1, 0, -1, -1, 0, -1, 0]],
[['worst', 'customer', 'service', 'ever', 'experienced', 'is', 'always', 'provided', 'and', 'rude'], [1, 1, 0, 0, 0, 2, 0, 0, 2, 0], [-1, -1, 0, 0, 0, -1, 0, -1, -1, -1]],
[['uncomfortable', 'and', 'rough', 'commute', 'experience', 'is', 'always', 'terrible', 'and', 'noisy'], [1, 0, 1, 1, 0, 2, 0, 0, 2, 0], [-1, 0, -1, -1, 0, -1, 0, -1, -1, -1]],
[['unsafe', 'and', 'unreliable', 'public', 'transportation', 'system', 'is', 'always', 'experienced', 'and', 'insecure', 'and', 'dangerous'], [1, 0, 1, 1, 0, 0, 2, 0, 0, 2, 0, 2, 0], [-1, 0, -1, -1, 0, 0, -1, 0, -1, -1, -1, -1, -1]],
[['inefficient', 'and', 'rude', 'service', 'from', 'the', 'staff', 'is', 'always', 'provided', 'and', 'unhelpful', 'and', 'slow'], [1, 0, 1, 1, 0, 0, 0, 2, 0, 0, 2, 0, 2, 0], [-1, 0, -1, -1, 0, 0, 0, -1, 0, -1, -1, -1, -1, -1]],
[['cleanliness', 'could', 'be', 'significantly', 'improved', 'and', 'is', 'a', 'concern'], [1, 0, 0, 0, 0, 2, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0]],
[['seats', 'are', 'just', 'okay', 'nothing', 'special', 'and', 'could', 'be', 'better'], [1, 0, 0, 0, 0, 0, 2, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
[['felt', 'reasonably', 'safe', 'during', 'the', 'day', 'and', 'early', 'evening'], [1, 0, 0, 0, 0, 0, 2, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0]],
[['average', 'customer', 'service', 'experience', 'overall', 'and', 'nothing', 'remarkable'], [1, 1, 0, 0, 0, 2, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0]],
[['facilities', 'are', 'adequate', 'for', 'basic', 'needs', 'and', 'functional'], [1, 0, 0, 0, 0, 0, 2, 0], [0, 0, 0, 0, 0, 0, 0, 0]],
[['access', 'is', 'passable', 'for', 'most', 'people', 'and', 'generally', 'convenient'], [1, 0, 0, 0, 0, 0, 2, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0]],
[['trains', 'are', 'sometimes', 'clean', 'but', 'not', 'always', 'and', 'can', 'be', 'crowded'], [1, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
[['ride', 'was', 'neither', 'comfortable', 'nor', 'uncomfortable', 'just', 'neutral', 'and', 'uneventful'], [1, 0, 0, 0, 0, 0, 0, 0, 2, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
[['safety', 'level', 'seems', 'average', 'during', 'peak', 'hours', 'and', 'generally', 'acceptable'], [1, 0, 0, 0, 0, 0, 0, 2, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
[['service', 'was', 'acceptable', 'but', 'could', 'be', 'better', 'and', 'more', 'proactive'], [1, 0, 0, 0, 0, 0, 0, 2, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
[['cleanliness', 'is', 'average', 'and', 'could', 'use', 'improvement', 'and', 'more', 'attention'], [1, 0, 0, 0, 0, 0, 0, 2, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
[['seats', 'are', 'moderately', 'comfortable', 'for', 'short', 'trips', 'and', 'reasonably', 'clean'], [1, 0, 0, 0, 0, 0, 0, 2, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
[['felt', 'generally', 'safe', 'but', 'would', 'prefer', 'more', 'security', 'and', 'visibility'], [1, 0, 0, 0, 0, 0, 0, 2, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
[['standard', 'customer', 'service', 'with', 'no', 'major', 'issues', 'and', 'generally', 'helpful'], [1, 1, 0, 0, 0, 0, 0, 2, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
[["dirty", "and", "contaminated", "air"], [1, 0, 1, 1], [-1, -1, -1, -1]],
[["frequent", "and", "unreliable", "service"], [1, 0, 1, 1], [-1, -1, -1, -1]],
[["uncomfortable", "and", "packed", "trains"], [1, 0, 1, 1], [-1, -1, -1, -1]],
[["unsafe", "and", "vulnerable", "environment"], [1, 0, 1, 1], [-1, -1, -1, -1]],
[["poor", "and", "unsympathetic", "response"], [1, 0, 1, 0], [-1, -1, -1, -1]],
[["old", "and", "worn-out", "interiors"], [1, 0, 1, 1], [-1, -1, -1, -1]],
[["difficult", "and", "tiring", "walks"], [1, 0, 1, 1], [-1, -1, -1, -1]],
[["noisy", "and", "disturbing", "sounds"], [1, 0, 1, 1], [-1, -1, -1, -1]],
[["expensive", "and", "unwarranted", "fees"], [1, 0, 1, 1, 1], [-1, -1, -1, -1, -1]],
[["inadequate", "resources", "and", "limited", "support"], [1, 1, 0, 1, 1], [-1, -1, -1, -1, -1]],
[['the', 'mrt', 'is', 'very', 'clean', 'and', 'well-maintained'], [0, 0, 0, 0, 1, 2, 1], [1, 1, 0, 0, 1, 1, 1]],
[['seats', 'are', 'comfortable', 'and', 'spacious'], [1, 0, 1, 2, 1], [1, 0, 1, 1, 1]],
[['felt', 'safe', 'and', 'secure', 'on', 'the', 'train'], [0, 1, 2, 1, 0, 0, 0], [1, 1, 1, 1, 0, 0, 0]],
[['good', 'customer', 'service', 'is', 'provided'], [1, 1, 0, 2, 1], [1, 1, 0, 1, 1]],
[['modern', 'and', 'clean', 'facilities', 'are', 'available'], [1, 0, 1, 1, 2, 1], [1, 0, 1, 1, 1, 1]],
[['easy', 'access', 'to', 'platforms', 'is', 'a', 'plus'], [1, 1, 0, 0, 2, 0, 1], [1, 1, 0, 0, 1, 0, 1]],
[['trains', 'are', 'always', 'clean', 'and', 'tidy', 'and', 'well-lit'], [1, 0, 0, 1, 2, 1, 2, 1], [1, 0, 0, 1, 1, 1, 1, 1]],
[['comfortable', 'temperature', 'and', 'smooth', 'ride', 'is', 'great'], [1, 1, 0, 1, 1, 2, 1], [1, 1, 0, 1, 1, 1, 1]],
[['very', 'safe', 'and', 'reliable', 'environment', 'is', 'ensured'], [1, 0, 1, 1, 0, 2, 1], [1, 0, 1, 1, 0, 1, 1]],
[['friendly', 'and', 'efficient', 'service', 'is', 'provided', 'always'], [1, 0, 1, 1, 2, 1, 0], [1, 0, 1, 1, 1, 1, 0]],
[["dirty", "and", "unhygienic", "restrooms"], [1, 0, 1, 1], [-1, -1, -1, -1]],
[["frequent", "and", "unforeseen", "stoppages"], [1, 0, 1, 1], [-1, -1, -1, -1]],
[["uncomfortable", "and", "claustrophobic", "space"], [1, 0, 1, 1], [-1, -1, -1, -1]],
[["unsafe", "and", "precarious", "conditions"], [1, 0, 1, 1], [-1, -1, -1, -1]],
[["poor", "and", "insensitive", "handling"], [1, 0, 1, 0], [-1, -1, -1, -1]],
[["old", "and", "decrepit", "structures"], [1, 0, 1, 1], [-1, -1, -1, -1]],
[["difficult", "and", "inconvenient", "connections"], [1, 0, 1, 1], [-1, -1, -1, -1]],
[["noisy", "and", "irritating", "noises"], [1, 0, 1, 1], [-1, -1, -1, -1]],
[["expensive", "and", "excessive", "tolls"], [1, 0, 1, 1, 1], [-1, -1, -1, -1, -1]],
[["inadequate", "maintenance", "and", "neglected", "facilities"], [1, 1, 0, 1, 1], [-1, -1, -1, -1, -1]],
[['cleanliness', 'is', 'well', 'maintained', 'and', 'evident'], [1, 0, 0, 1, 2, 1], [1, 0, 0, 1, 1, 1]],
[['seats', 'are', 'very', 'comfortable', 'and', 'spacious', 'and', 'clean'], [1, 0, 0, 1, 2, 1, 2, 1], [1, 0, 0, 1, 1, 1, 1, 1]],
[['felt', 'safe', 'traveling', 'alone', 'at', 'night', 'and', 'secure'], [1, 0, 0, 0, 0, 0, 2, 1], [1, 0, 0, 0, 0, 0, 1, 1]],
[['excellent', 'and', 'prompt', 'service', 'is', 'delivered'], [1, 0, 1, 1, 2, 1], [1, 0, 1, 1, 1, 1]],
[['modern', 'and', 'clean', 'facilities', 'are', 'readily', 'available'], [1, 0, 1, 1, 2, 0, 1], [1, 0, 1, 1, 1, 0, 1]],
[['good', 'access', 'for', 'families', 'with', 'strollers', 'is', 'important'], [1, 1, 0, 0, 0, 0, 2, 1], [1, 1, 0, 0, 0, 0, 1, 1]],
[['trains', 'are', 'spotlessly', 'clean', 'and', 'well-maintained', 'and', 'comfortable'], [1, 0, 0, 1, 2, 1, 2, 1], [1, 0, 0, 1, 1, 1, 1, 1]],
[['comfortable', 'and', 'quiet', 'journey', 'is', 'appreciated'], [1, 0, 1, 1, 2, 1], [1, 0, 1, 1, 1, 1]],
[['fast', 'and', 'efficient', 'service', 'is', 'always', 'provided'], [1, 0, 1, 1, 2, 0, 1], [1, 0, 1, 1, 1, 0, 1]],
[['cleanliness', 'is', 'consistently', 'high', 'and', 'noticeable'], [1, 0, 0, 1, 2, 1], [1, 0, 0, 1, 1, 1]],
[['comfortable', 'seats', 'with', 'good', 'back', 'support', 'are', 'provided'], [1, 1, 0, 1, 0, 0, 2, 1], [1, 1, 0, 1, 0, 0, 1, 1]],
[['felt', 'very', 'safe', 'and', 'secure', 'throughout', 'the', 'journey', 'and', 'comfortable'], [1, 0, 0, 1, 1, 0, 0, 0, 2, 1], [1, 0, 0, 1, 1, 0, 0, 0, 1, 1]],
[['friendly', 'and', 'helpful', 'staff', 'are', 'present', 'everywhere'], [1, 0, 1, 1, 2, 1, 0], [1, 0, 1, 1, 1, 1, 0]],
[['modern', 'facilities', 'and', 'excellent', 'maintenance', 'are', 'provided', 'regularly'], [1, 1, 0, 1, 0, 2, 1, 0], [1, 1, 0, 1, 0, 1, 0, 0]],
[['easy', 'access', 'to', 'all', 'train', 'lines', 'is', 'guaranteed'], [1, 1, 0, 0, 0, 0, 2, 1], [1, 1, 0, 0, 0, 0, 1, 1]],
[['trains', 'are', 'always', 'clean', 'and', 'well-organized', 'and', 'on-time'], [1, 0, 0, 1, 2, 1, 2, 1], [1, 0, 0, 1, 1, 1, 1, 1]],
[['comfortable', 'and', 'smooth', 'commute', 'is', 'always', 'guaranteed'], [1, 0, 1, 1, 2, 0, 1], [1, 0, 1, 1, 1, 0, 1]],
[['the', 'MRT', 'provides', 'a', 'very', 'clean', 'and', 'efficient', 'service'], [0, 0, 0, 0, 0, 1, 2, 1, 1], [1, 1, 0, 0, 0, 1, 1, 1, 1]],
[['seats', 'are', 'comfortable', 'even', 'during', 'long', 'journeys'], [1, 0, 1, 0, 0, 0, 0], [1, 0, 1, 1, 1, 1, 1]],
[["dirty", "and", "unsanitary", "conditions"], [1, 0, 1, 1], [-1, -1, -1, -1]],
[["frequent", "and", "unpredictable", "delays"], [1, 0, 1, 1], [-1, -1, -1, -1]],
[["uncomfortable", "and", "overcrowded", "trains"], [1, 0, 1, 1], [-1, -1, -1, -1]],
[["unsafe", "and", "risky", "behavior"], [1, 0, 1, 1], [-1, -1, -1, -1]],
[["poor", "and", "discourteous", "treatment"], [1, 0, 1, 0], [-1, -1, -1, -1]],
[["old", "and", "worn-out", "equipment"], [1, 0, 1, 1], [-1, -1, -1, -1]],
[["difficult", "and", "complicated", "fare", "system"], [1, 0, 1, 1, 1], [-1, -1, -1, -1, -1]],
[["noisy", "and", "disagreeable", "sounds"], [1, 0, 1, 1], [-1, -1, -1, -1]],
[["expensive", "and", "unjustified", "penalties"], [1, 0, 1, 1, 1], [-1, -1, -1, -1, -1]],
[["inadequate", "space", "and", "lack", "of", "room"], [1, 1, 0, 1, 0, 1], [-1, -1, -1, -1, -1, -1]],
[['i', 'felt', 'safe', 'and', 'secure', 'traveling', 'at', 'night'], [0, 0, 1, 2, 1, 0, 0, 0], [1, 0, 1, 1, 1, 1, 1, 1]],
[['customer', 'service', 'is', 'consistently', 'good', 'and', 'helpful'], [1, 1, 0, 0, 1, 2, 1], [1, 1, 0, 0, 1, 1, 1]],
[['modern', 'facilities', 'and', 'clean', 'environment', 'are', 'a', 'priority'], [1, 1, 0, 1, 1, 2, 0, 1], [1, 1, 0, 1, 1, 1, 0, 1]],
[['easy', 'access', 'and', 'clear', 'signage', 'make', 'navigation', 'simple'], [1, 1, 2, 1, 0, 0, 0, 0], [1, 1, 1, 1, 0, 0, 0, 0]],
[['trains', 'are', 'always', 'clean', 'well-maintained', 'and', 'on', 'time'], [1, 0, 0, 1, 2, 1, 0, 0], [1, 0, 0, 1, 1, 1, 1, 1]],
[['comfortable', 'ride', 'and', 'pleasant', 'temperature', 'are', 'appreciated'], [1, 1, 0, 1, 1, 2, 1], [1, 1, 0, 1, 1, 1, 1]],
[['a', 'very', 'safe', 'and', 'reliable', 'mode', 'of', 'transportation'], [0, 0, 1, 2, 1, 0, 0, 0], [1, 0, 1, 1, 1, 1, 1, 1]],
[['friendly', 'and', 'efficient', 'service', 'from', 'staff', 'is', 'consistent'], [1, 0, 1, 1, 0, 0, 0, 0], [1, 0, 1, 1, 0, 1, 1, 1]],
[['high', 'level', 'of', 'cleanliness', 'is', 'maintained', 'throughout'], [1, 0, 0, 1, 2, 1, 0], [1, 0, 0, 1, 1, 1, 1]],
[['seats', 'are', 'comfortable', 'spacious', 'and', 'clean', 'which', 'is', 'great'], [1, 0, 1, 2, 1, 1, 0, 0, 1], [1, 0, 1, 1, 1, 1, 0, 0, 1]],
[['i', 'always', 'feel', 'safe', 'and', 'secure', 'traveling', 'on', 'the', 'MRT'], [0, 0, 0, 1, 2, 1, 0, 0, 0, 0], [1, 0, 0, 1, 1, 1, 1, 0, 0, 0]],
[['excellent', 'service', 'and', 'prompt', 'response', 'from', 'staff'], [1, 1, 0, 1, 1, 0, 0], [1, 1, 0, 1, 1, 1, 1]],
[["dirty", "and", "littered", "platforms"], [1, 0, 1, 1], [-1, -1, -1, -1]],
[["frequent", "and", "disruptive", "announcements"], [1, 0, 1, 1], [-1, -1, -1, -1]],
[["uncomfortable", "and", "stuffy", "atmosphere"], [1, 0, 1, 1], [-1, -1, -1, -1]],
[["unsafe", "and", "hazardous", "gaps"], [1, 0, 1, 1], [-1, -1, -1, -1]],
[["poor", "and", "unresponsive", "management"], [1, 0, 1, 0], [-1, -1, -1, -1]],
[["old", "and", "obsolete", "technology"], [1, 0, 1, 1], [-1, -1, -1, -1]],
[["difficult", "and", "lengthy", "queues"], [1, 0, 1, 1], [-1, -1, -1, -1]],
[["noisy", "and", "squealing", "wheels"], [1, 0, 1, 1], [-1, -1, -1, -1]],
[["expensive", "and", "inflated", "concession", "prices"], [1, 0, 1, 1, 1], [-1, -1, -1, -1, -1]],
[["inadequate", "signage", "and", "poor", "visibility"], [1, 1, 0, 1, 1], [-1, -1, -1, -1, -1]],
[['modern', 'facilities', 'and', 'easy', 'accessibility', 'are', 'key', 'features'], [1, 1, 0, 1, 1, 2, 0, 1], [1, 1, 0, 1, 1, 1, 0, 1]],
[['good', 'access', 'for', 'passengers', 'with', 'disabilities', 'is', 'provided'], [1, 1, 0, 0, 0, 0, 2, 1], [1, 1, 0, 0, 0, 0, 1, 1]],
[['trains', 'are', 'consistently', 'clean', 'well-maintained', 'and', 'comfortable'], [1, 0, 0, 1, 2, 1, 1], [1, 0, 0, 1, 1, 1, 1]],
[['a', 'comfortable', 'and', 'enjoyable', 'journey', 'is', 'guaranteed', 'every', 'time'], [0, 1, 0, 1, 1, 2, 1, 0, 0], [1, 1, 0, 1, 1, 1, 1, 1, 1]],
[['the', 'mrt', 'system', 'is', 'very', 'safe', 'reliable', 'and', 'efficient'], [0, 0, 0, 0, 0, 1, 2, 1, 1], [1, 1, 1, 0, 0, 1, 1, 1, 1]],
[['fast', 'efficient', 'and', 'friendly', 'service', 'is', 'always', 'appreciated'], [1, 1, 2, 1, 1, 2, 0, 1], [1, 1, 1, 1, 1, 1, 1, 1]],
[['high', 'standards', 'of', 'cleanliness', 'are', 'always', 'maintained'], [1, 0, 0, 1, 2, 0, 1], [1, 0, 0, 1, 1, 1, 1]],
[['seats', 'are', 'comfortable', 'with', 'ample', 'legroom', 'and', 'good', 'support'], [1, 0, 1, 0, 0, 0, 2, 1, 0], [1, 0, 1, 1, 1, 1, 1, 1, 1]],
[['poor', 'air', 'circulation'], [1, 1, 2], [-1, -1, -1]],
[['limited', 'and', 'inconvenient', 'access'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['frequent', 'and', 'unexpected', 'breakdowns'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['lack', 'of', 'clear', 'signage'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['inadequate', 'and', 'uncomfortable', 'waiting', 'areas'], [1, 0, 1, 2, 0, 0], [-1, -1, -1, -1, -1, -1]],
[['dirty', 'and', 'poorly', 'maintained', 'facilities'], [1, 0, 0, 1, 2], [-1, -1, -1, -1, -1]],
[['overcrowded', 'and', 'noisy', 'platforms'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['slow', 'and', 'unreliable', 'service'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['poor', 'and', 'inconsistent', 'communication'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['lack', 'of', 'safety', 'measures'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['inadequate', 'and', 'dirty', 'ventilation'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['long', 'and', 'unpredictable', 'delays'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['limited', 'and', 'inaccessible', 'elevators'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['unfriendly', 'and', 'unhelpful', 'staff'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['dirty', 'and', 'smelly', 'restrooms'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['i', 'feel', 'very', 'safe', 'and', 'secure', 'throughout', 'my', 'commute', 'on', 'the', 'mrt'], [0, 0, 0, 1, 2, 1, 0, 0, 0, 0, 0, 0], [1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
[['staff', 'are', 'friendly', 'helpful', 'and', 'provide', 'excellent', 'assistance'], [1, 0, 1, 2, 1, 0, 1, 1], [1, 0, 1, 1, 1, 1, 1, 1]],
[['modern', 'facilities', 'well-maintained', 'trains', 'and', 'punctual', 'service', 'are', 'guaranteed'], [1, 1, 1, 1, 0, 1, 1, 2, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1]],
[['easy', 'access', 'to', 'all', 'parts', 'of', 'the', 'city', 'is', 'a', 'major', 'plus'], [1, 1, 0, 0, 0, 0, 0, 0, 2, 0, 0, 1], [1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1]],
[['trains', 'are', 'always', 'clean', 'comfortable', 'and', 'arrive', 'on', 'time', 'consistently'], [1, 0, 0, 1, 2, 1, 0, 0, 0, 0], [1, 0, 0, 1, 1, 1, 1, 1, 1, 1]],
[['a', 'smooth', 'comfortable', 'and', 'reliable', 'commute', 'is', 'always', 'ensured'], [0, 1, 2, 1, 1, 1, 2, 0, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1]],
[['the', 'mrt', 'offers', 'a', 'clean', 'safe', 'and', 'efficient', 'way', 'to', 'travel'], [0, 0, 0, 0, 1, 2, 1, 1, 0, 0, 0], [1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1]],
[['comfortable', 'seats', 'and', 'a', 'smooth', 'ride', 'make', 'the', 'journey', 'enjoyable'], [1, 1, 0, 0, 1, 1, 0, 0, 0, 1], [1, 1, 0, 0, 1, 1, 1, 0, 0, 1]],
[['passengers', 'feel', 'safe', 'and', 'secure', 'with', 'the', 'visible', 'security', 'presence'], [1, 0, 1, 2, 1, 0, 0, 0, 0, 0], [1, 0, 1, 1, 1, 1, 0, 0, 0, 1]],
[['the', 'mrt', 'is', 'not', 'very', 'clean'], [0, 0, 0, 0, 0, 1], [-1, -1, 0, 0, 0, -1]],
[['seats', 'are', 'uncomfortable', 'and', 'worn', 'out'], [1, 0, 1, 0, 1, 0], [-1, 0, -1, 0, -1, 0]],
[['felt', 'unsafe', 'on', 'the', 'train', 'at', 'night'], [0, 1, 0, 0, 0, 0, 0], [-1, -1, 0, 0, 0, 0, 0]],
[['poor', 'customer', 'service', 'experience'], [1, 1, 0, 0], [-1, -1, 0, 0]],
[['dirty', 'and', 'old', 'facilities', 'in', 'poor', 'condition'], [1, 0, 1, 1, 0, 0, 0], [-1, 0, -1, -1, 0, 0, 0]],
[['difficult', 'access', 'to', 'platforms', 'for', 'elderly'], [1, 1, 0, 0, 0, 0], [-1, -1, 0, 0, 0, 0]],
[['trains', 'are', 'rarely', 'clean', 'and', 'well-maintained'], [1, 0, 0, 1, 0, 1], [-1, 0, 0, -1, 0, -1]],
[['good', 'customer', 'service', 'and', 'helpful', 'staff', 'enhance', 'the', 'overall', 'experience'], [1, 1, 0, 2, 1, 1, 0, 0, 0, 1], [1, 1, 0, 1, 1, 1, 1, 0, 0, 1]],
[['modern', 'well-maintained', 'facilities', 'contribute', 'to', 'a', 'pleasant', 'environment'], [1, 1, 1, 1, 0, 0, 0, 1], [1, 1, 1, 1, 1, 0, 1, 1]],
[['easy', 'access', 'to', 'stations', 'and', 'platforms', 'is', 'a', 'significant', 'advantage'], [1, 1, 0, 0, 0, 0, 2, 0, 0, 1], [1, 1, 0, 0, 0, 0, 1, 0, 1, 1]],
[['trains', 'are', 'always', 'clean', 'on', 'time', 'and', 'provide', 'a', 'comfortable', 'ride'], [1, 0, 0, 1, 0, 0, 2, 0, 0, 1, 1], [1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1]],
[['a', 'comfortable', 'and', 'smooth', 'ride', 'is', 'guaranteed', 'making', 'the', 'commute', 'enjoyable'], [0, 1, 0, 1, 1, 2, 1, 0, 0, 1, 1], [1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1]],
[['the', 'mrt', 'system', 'ensures', 'a', 'safe', 'and', 'reliable', 'journey', 'for', 'all', 'passengers'], [0, 0, 0, 0, 0, 1, 2, 1, 1, 0, 0, 0], [1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1]],
[['fast', 'efficient', 'and', 'friendly', 'service', 'from', 'staff', 'makes', 'the', 'experience', 'positive'], [1, 1, 2, 1, 1, 0, 0, 0, 0, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1]],
[['comfortable', 'seats', 'ample', 'legroom', 'and', 'good', 'back', 'support', 'are', 'provided', 'throughout'], [1, 1, 1, 2, 1, 1, 0, 0, 2, 1, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
[["dirty", "and", "infested", "areas"], [1, 0, 1, 1], [-1, -1, -1, -1]],
[["frequent", "and", "disorganized", "crowds"], [1, 0, 1, 1], [-1, -1, -1, -1]],
[["uncomfortable", "and", "stifling", "heat"], [1, 0, 1, 1], [-1, -1, -1, -1]],
[["unsafe", "and", "exposed", "conditions"], [1, 0, 1, 1], [-1, -1, -1, -1]],
[["poor", "and", "indifferent", "attitude"], [1, 0, 1, 0], [-1, -1, -1, -1]],
[["old", "and", "decaying", "structures"], [1, 0, 1, 1], [-1, -1, -1, -1]],
[["difficult", "and", "complicated", "routes"], [1, 0, 1, 1], [-1, -1, -1, -1]],
[["noisy", "and", "unpleasant", "odors"], [1, 0, 1, 1], [-1, -1, -1, -1]],
[["expensive", "and", "unnecessary", "charges"], [1, 0, 1, 1, 1], [-1, -1, -1, -1, -1]],
[["inadequate", "staffing", "and", "long", "waits"], [1, 1, 0, 1, 1], [-1, -1, -1, -1, -1]],
[['i', 'feel', 'safe', 'and', 'secure', 'traveling', 'on', 'the', 'mrt', 'at', 'any', 'time'], [0, 0, 1, 2, 1, 0, 0, 0, 0, 0, 0, 0], [1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1]],
[['the', 'staff', 'are', 'friendly', 'helpful', 'and', 'always', 'ready', 'to', 'assist', 'passengers'], [0, 1, 0, 1, 2, 1, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
[['modern', 'facilities', 'and', 'regular', 'maintenance', 'ensure', 'a', 'pleasant', 'journey', 'for', 'everyone'], [1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0], [1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1]],
[['easy', 'access', 'to', 'all', 'parts', 'of', 'the', 'city', 'is', 'made', 'possible', 'by', 'the', 'efficient', 'mrt', 'system'], [1, 1, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 1, 1, 0], [1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1]],
[['trains', 'are', 'consistently', 'clean', 'comfortable', 'and', 'arrive', 'on', 'schedule', 'without', 'delays'], [1, 0, 0, 1, 2, 1, 0, 0, 0, 0, 0], [1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]],
[['a', 'smooth', 'comfortable', 'and', 'safe', 'commute', 'is', 'always', 'guaranteed', 'for', 'all', 'travelers'], [0, 1, 2, 1, 1, 1, 2, 0, 1, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
[['the', 'MRT', 'is', 'very', 'clean'], [0, 0, 0, 0, 1], [1, 1, 0, 0, 1]],
[['seats', 'are', 'comfortable', 'and', 'plush'], [1, 0, 1, 0, 1], [1, 0, 1, 0, 1]],
[['felt', 'safe', 'with', 'security', 'present'], [0, 1, 0, 1, 0], [1, 1, 0, 1, 0]],
[['good', 'customer', 'service', 'overall'], [1, 1, 0, 0], [1, 1, 0, 0]],
[['modern', 'and', 'well-maintained', 'facilities'], [1, 0, 1, 1], [1, 0, 1, 1]],
[['easy', 'access', 'for', 'all', 'passengers'], [1, 1, 0, 0, 0], [1, 1, 0, 0, 0]],
[['poorly-lit', 'and', 'dark', 'stations'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['run-down', 'and', 'shabby', 'buildings'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['negligent', 'and', 'careless', 'handling'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['trash-filled', 'and', 'littered', 'areas'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['sardine-can', 'and', 'packed', 'trains'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['vulnerable', 'and', 'unguarded', 'points'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['dilapidated', 'and', 'decrepit', 'structures'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['delayed', 'and', 'disrupted', 'service'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['polluted', 'and', 'foul-smelling', 'air'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['hard', 'and', 'stiff', 'seats'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['rough', 'and', 'jerky', 'ride'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['overcrowded', 'and', 'congested', 'platforms'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['missing', 'and', 'non-existent', 'facilities'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['appalling', 'and', 'abysmal', 'treatment'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['stained', 'and', 'unclean', 'surfaces'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['stuffy', 'and', 'poorly-ventilated', 'coaches'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['faulty', 'and', 'defective', 'equipment'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['run-down', 'and', 'obsolete', 'systems'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['dismissive', 'and', 'unfriendly', 'attitude'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['trains', 'are', 'always', 'clean', 'and', 'tidy'], [1, 0, 0, 1, 0, 1], [1, 0, 0, 1, 0, 1]],
[['comfortable', 'temperature', 'and', 'smooth', 'ride'], [1, 1, 0, 1, 1], [1, 1, 0, 1, 1]],
[['very', 'safe', 'and', 'secure', 'environment'], [1, 0, 1, 1, 0], [1, 0, 1, 1, 0]],
[['friendly', 'and', 'efficient', 'service', 'provided'], [1, 0, 1, 1, 0], [1, 0, 1, 1, 0]],
[['cleanliness', 'is', 'a', 'top', 'priority'], [1, 0, 0, 1, 0], [1, 0, 0, 1, 0]],
[['seats', 'are', 'very', 'comfortable', 'and', 'spacious'], [1, 0, 0, 1, 0, 1], [1, 0, 0, 1, 0, 1]],
[['felt', 'safe', 'traveling', 'alone', 'at', 'night'], [1, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0]],
[['excellent', 'and', 'prompt', 'service'], [1, 0, 1, 1], [1, 0, 1, 1]],
[['modern', 'and', 'clean', 'facilities', 'available'], [1, 0, 1, 1, 0], [1, 0, 1, 1, 0]],
[['good', 'access', 'for', 'families', 'with', 'strollers'], [1, 1, 0, 0, 0, 0], [1, 1, 0, 0, 0, 0]],
[['trains', 'are', 'spotlessly', 'clean', 'and', 'well-maintained'], [1, 0, 0, 1, 0, 1], [1, 0, 0, 1, 0, 1]],
[['comfortable', 'and', 'quiet', 'journey'], [1, 0, 1, 1], [1, 0, 1, 1]],
[['very', 'safe', 'and', 'reliable', 'transport'], [1, 0, 1, 1, 0], [1, 0, 1, 1, 0]],
[['fast', 'and', 'efficient', 'service', 'always'], [1, 0, 1, 1, 0], [1, 0, 1, 1, 0]],
[['cleanliness', 'is', 'consistently', 'high'], [1, 0, 0, 1], [1, 0, 0, 1]],
[["dirty", "and", "stained", "upholstery"], [1, 0, 1, 1], [-1, -1, -1, -1]],
[["frequent", "and", "prolonged", "stoppages"], [1, 0, 1, 1], [-1, -1, -1, -1]],
[["uncomfortable", "and", "cramped", "conditions"], [1, 0, 1, 1], [-1, -1, -1, -1]],
[["unsafe", "and", "reckless", "driving"], [1, 0, 1, 1], [-1, -1, -1, -1]],
[["poor", "and", "inattentive", "service"], [1, 0, 1, 0], [-1, -1, -1, -1]],
[["old", "and", "dilapidated", "carriages"], [1, 0, 1, 1], [-1, -1, -1, -1]],
[["difficult", "and", "time-consuming", "transfers"], [1, 0, 1, 1], [-1, -1, -1, -1]],
[["noisy", "and", "rattling", "trains"], [1, 0, 1, 1], [-1, -1, -1, -1]],
[["expensive", "and", "exorbitant", "parking", "fees"], [1, 0, 1, 1, 1], [-1, -1, -1, -1, -1]],
[["inadequate", "information", "and", "confusing", "directions"], [1, 1, 0, 1, 1], [-1, -1, -1, -1, -1]],
[['comfortable', 'seats', 'with', 'good', 'back', 'support'], [1, 1, 0, 1, 0, 0], [1, 1, 0, 1, 0, 0]],
[['felt', 'very', 'safe', 'and', 'secure', 'throughout'], [1, 0, 0, 1, 1, 0], [1, 0, 0, 1, 1, 0]],
[['friendly', 'and', 'helpful', 'staff', 'everywhere'], [1, 0, 1, 1, 0], [1, 0, 1, 1, 0]],
[['modern', 'facilities', 'and', 'excellent', 'maintenance'], [1, 1, 0, 1, 0], [1, 1, 0, 1, 0]],
[['easy', 'access', 'to', 'all', 'train', 'lines'], [1, 1, 0, 0, 0, 0], [1, 1, 0, 0, 0, 0]],
[['trains', 'are', 'always', 'clean', 'and', 'well-organized'], [1, 0, 0, 1, 0, 1], [1, 0, 0, 1, 0, 1]],
[['comfortable', 'and', 'smooth', 'commute'], [1, 0, 1, 1], [1, 0, 1, 1]],
[['safe', 'and', 'reliable', 'public', 'transportation'], [1, 0, 1, 1, 0], [1, 0, 1, 1, 0]],
[['efficient', 'and', 'courteous', 'service', 'provided', 'by', 'staff'], [1, 0, 1, 1, 0, 0, 0], [1, 0, 1, 1, 0, 0, 0]],
[['high', 'level', 'of', 'cleanliness', 'observed'], [1, 1, 0, 0, 0], [1, 1, 0, 0, 0]],
[['smelly', 'and', 'sticky', 'handrails'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['noisy', 'and', 'bumpy', 'journey'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['insecure', 'and', 'exposed', 'areas'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['unavailable', 'and', 'unusable', 'toilets'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['incompetent', 'and', 'unprofessional', 'conduct'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['dusty', 'and', 'grimy', 'windows'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['hot', 'and', 'stale', 'air'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['slippery', 'and', 'uneven', 'steps'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['damaged', 'and', 'deteriorated', 'seats'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['unfriendly', 'and', 'unsympathetic', 'response'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['messy', 'and', 'unhygienic', 'conditions'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['suffocating', 'and', 'stifling', 'atmosphere'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['comfortable', 'and', 'well-maintained', 'seats', 'always'], [1, 0, 1, 1, 0], [1, 0, 1, 1, 0]],
[['felt', 'safe', 'and', 'protected', 'throughout', 'the', 'journey'], [1, 0, 1, 1, 0, 0, 0], [1, 0, 1, 1, 0, 0, 0]],
[['excellent', 'and', 'fast', 'service', 'delivery'], [1, 0, 1, 1, 0], [1, 0, 1, 1, 0]],
[['modern', 'and', 'well-equipped', 'facilities', 'available', 'for', 'everyone'], [1, 0, 1, 1, 0, 0, 0], [1, 0, 1, 1, 0, 0, 0]],
[['easy', 'and', 'convenient', 'access', 'to', 'all', 'areas', 'of', 'the', 'station'], [1, 0, 1, 1, 0, 0, 0, 0, 0, 0], [1, 0, 1, 1, 0, 0, 0, 0, 0, 0]],
[['trains', 'are', 'consistently', 'clean', 'and', 'well-maintained'], [1, 0, 0, 1, 0, 1], [1, 0, 0, 1, 0, 1]],
[['comfortable', 'and', 'enjoyable', 'commute'], [1, 0, 1, 1], [1, 0, 1, 1]],
[['safe', 'and', 'secure', 'travel', 'experience'], [1, 0, 1, 1, 0], [1, 0, 1, 1, 0]],
[['efficient', 'and', 'professional', 'service', 'provided'], [1, 0, 1, 1, 0], [1, 0, 1, 1, 0]],
[['superb', 'cleanliness', 'standards', 'maintained'], [1, 1, 0, 0], [1, 1, 0, 0]],
[['very', 'comfortable', 'and', 'plush', 'seats'], [1, 0, 1, 0, 1], [1, 0, 1, 0, 1]],
[['felt', 'extremely', 'safe', 'and', 'secure', 'always'], [1, 0, 1, 0, 1, 0], [1, 0, 1, 0, 1, 0]],
[['outstanding', 'customer', 'service', 'experience'], [1, 1, 0, 0], [1, 1, 0, 0]],
[['highly', 'accessible', 'stations', 'for', 'everyone'], [1, 1, 0, 0, 0], [1, 1, 0, 0, 0]],
[['trains', 'are', 'always', 'immaculate', 'and', 'well-kept'], [1, 0, 0, 1, 0, 1], [1, 0, 0, 1, 0, 1]],
[['comfortable', 'and', 'relaxing', 'journey', 'always'], [1, 0, 1, 1, 0], [1, 0, 1, 1, 0]],
[['safe', 'and', 'reliable', 'public', 'transportation', 'system'], [1, 0, 1, 1, 0, 0], [1, 0, 1, 1, 0, 0]],
[['efficient', 'and', 'timely', 'service', 'delivery', 'always'], [1, 0, 1, 1, 0, 0], [1, 0, 1, 1, 0, 0]],
[['uncomfortable', 'and', 'noisy', 'commute'], [1, 0, 1, 1], [-1, 0, -1, -1]],
[['felt', 'unsafe', 'and', 'insecure', 'throughout', 'the', 'journey'], [1, 0, 1, 1, 0, 0, 0], [-1, 0, -1, -1, 0, 0, 0]],
[['slow', 'and', 'inefficient', 'service', 'delivery', 'always'], [1, 0, 1, 1, 0, 0], [-1, 0, -1, -1, 0, 0]],
[['lack', 'of', 'cleanliness', 'is', 'a', 'major', 'issue'], [1, 1, 0, 0, 0, 0, 0], [-1, -1, 0, 0, 0, 0, 0]],
[['very', 'uncomfortable', 'and', 'hard', 'seats'], [1, 0, 1, 0, 1], [-1, 0, -1, 0, -1]],
[['felt', 'unsafe', 'traveling', 'alone', 'at', 'night', 'frequently'], [0, 1, 0, 0, 0, 0, 0], [0, -1, 0, 0, 0, 0, 0]],
[['terrible', 'customer', 'service', 'provided', 'by', 'staff'], [1, 1, 0, 0, 0, 0], [-1, -1, 0, 0, 0, 0]],
[['old', 'and', 'dirty', 'facilities', 'in', 'a', 'state', 'of', 'disrepair'], [1, 0, 1, 1, 0, 0, 0, 0, 0], [-1, 0, -1, -1, 0, 0, 0, 0, 0]],
[['poor', 'access', 'for', 'disabled', 'passengers'], [1, 1, 0, 0, 0], [-1, -1, 0, 0, 0]],
[['trains', 'are', 'almost', 'never', 'clean', 'and', 'well-maintained'], [1, 0, 0, 0, 0, 0, 0], [-1, 0, 0, -1, -1, 0, -1]],
[['uncomfortable', 'and', 'bumpy', 'commute', 'experience'], [1, 0, 1, 1, 0], [-1, 0, -1, -1, 0]],
[['very', 'unsafe', 'and', 'unreliable', 'public', 'transport'], [1, 0, 1, 1, 0, 0], [-1, 0, -1, -1, 0, 0]],
[['slow', 'and', 'unhelpful', 'service', 'from', 'the', 'personnel'], [1, 0, 1, 1, 0, 0, 0], [-1, 0, -1, -1, 0, 0, 0]],
[['extremely', 'dirty', 'and', 'unhygienic', 'conditions'], [1, 1, 0, 0, 0], [-1, -1, 0, 0, 0]],
[['highly', 'uncomfortable', 'and', 'hard', 'seats', 'always'], [1, 1, 1, 0, 1, 0], [-1, -1, -1, 0, -1, 0]],
[['felt', 'extremely', 'unsafe', 'and', 'vulnerable'], [1, 0, 1, 0, 1], [-1, 0, -1, 0, -1]],
[['worst', 'customer', 'service', 'ever', 'experienced'], [1, 1, 0, 0, 0], [-1, -1, 0, 0, 0]],
[['dilapidated', 'and', 'filthy', 'facilities', 'in', 'a', 'terrible', 'state'], [1, 0, 1, 1, 0, 0, 0, 0], [-1, 0, -1, -1, 0, 0, 0, 0]],
[['inaccessible', 'stations', 'for', 'many', 'passengers'], [1, 1, 0, 0, 0], [-1, -1, 0, 0, 0]],
[['trains', 'are', 'consistently', 'filthy', 'and', 'poorly', 'maintained'], [1, 0, 0, 0, 0, 0, 0], [-1, 0, 0, -1, -1, 0, -1]],
[['uncomfortable', 'and', 'rough', 'commute', 'experience', 'always'], [1, 0, 1, 1, 0, 0], [-1, 0, -1, -1, 0, 0]],
[['unsafe', 'and', 'unreliable', 'public', 'transportation', 'system', 'overall'], [1, 0, 1, 0, 0, 0, 0], [-1, 0, -1, -1, 0, 0, 0]],
[['inefficient', 'and', 'rude', 'service', 'from', 'the', 'staff', 'always'], [1, 0, 1, 1, 0, 0, 0, 0], [-1, 0, -1, -1, 0, 0, 0, 0]],
[['cleanliness', 'could', 'be', 'significantly', 'improved'], [1, 0, 0, 0, 0], [0, 0, 0, 0, 0]],
[['seats', 'are', 'just', 'okay', 'nothing', 'special'], [1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]],
[['felt', 'reasonably', 'safe', 'during', 'the', 'day'], [1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]],
[['average', 'customer', 'service', 'experience', 'overall'], [1, 1, 0, 0, 0], [0, 0, 0, 0, 0]],
[['facilities', 'are', 'adequate', 'for', 'basic', 'needs'], [1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]],
[['access', 'is', 'passable', 'for', 'most', 'people'], [1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]],
[['trains', 'are', 'sometimes', 'clean', 'but', 'not', 'always'], [1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]],
[['ride', 'was', 'neither', 'comfortable', 'nor', 'uncomfortable', 'just', 'neutral'], [1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0]],
[['safety', 'level', 'seems', 'average', 'during', 'peak', 'hours'], [1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]],
[['service', 'was', 'acceptable', 'but', 'could', 'be', 'better'], [1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]],
[['cleanliness', 'is', 'average', 'and', 'could', 'use', 'improvement'], [1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]],
[["dirty", "and", "sticky", "surfaces"], [1, 0, 1, 1], [-1, -1, -1, -1]],
[["frequent", "and", "unexpected", "halts"], [1, 0, 1, 1], [-1, -1, -1, -1]],
[["uncomfortable", "and", "rigid", "seats"], [1, 0, 1, 1], [-1, -1, -1, -1]],
[["unsafe", "and", "chaotic", "boarding"], [1, 0, 1, 1], [-1, -1, -1, -1]],
[["poor", "and", "dismissive", "attitude"], [1, 0, 1, 0], [-1, -1, -1, -1]],
[["old", "and", "rundown", "facilities"], [1, 0, 1, 1], [-1, -1, -1, -1]],
[["difficult", "and", "confusing", "interchanges"], [1, 0, 1, 1], [-1, -1, -1, -1]],
[["noisy", "and", "jarring", "brakes"], [1, 0, 1, 1], [-1, -1, -1, -1]],
[["expensive", "and", "unreasonable", "surcharges"], [1, 0, 1, 1, 1], [-1, -1, -1, -1, -1]],
[["inadequate", "security", "and", "lack", "of", "presence"], [1, 1, 0, 1, 0, 1], [-1, -1, -1, -1, -1, -1]],
[['seats', 'are', 'moderately', 'comfortable', 'for', 'short', 'trips'], [1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]],
[['felt', 'generally', 'safe', 'but', 'would', 'prefer', 'more', 'security'], [1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0]],
[['standard', 'customer', 'service', 'with', 'no', 'major', 'issues'], [1, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]],
[['facilities', 'are', 'decent', 'but', 'could', 'be', 'more', 'modern'], [1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0]],
[['access', 'is', 'fair', 'but', 'could', 'be', 'improved', 'for', 'disabled'], [1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0]],
[['trains', 'are', 'usually', 'clean', 'but', 'can', 'get', 'dirty', 'sometimes'], [1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0]],
[['ride', 'was', 'reasonably', 'comfortable', 'for', 'the', 'duration'], [1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]],
[['safety', 'is', 'satisfactory', 'overall', 'with', 'room', 'for', 'improvement'], [1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0]],
[['service', 'is', 'adequate', 'but', 'staff', 'could', 'be', 'more', 'helpful'], [1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0]],
[['cleanliness', 'was', 'excellent', 'and', 'impressive'], [1, 0, 0, 2, 0], [1, 0, 0, 1, 0]],
[['comfort', 'during', 'the', 'ride', 'was', 'amazing'], [1, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0]],
[['safety', 'level', 'felt', 'low', 'at', 'night'], [1, 0, 0, 0, 2, 0], [-1, 0, 0, -1, -1, 0]],
[['facilities', 'are', 'not', 'well', 'maintained'], [1, 0, 0, 0, 2], [-1, 0, 0, 0, -1]],
[['service', 'was', 'average', 'but', 'acceptable'], [1, 0, 0, 0, 2], [0, 0, 0, 0, 0]],
[['the', 'cleanliness', 'in', 'toilets', 'was', 'disgusting'], [0, 1, 0, 2, 0, 0], [0, -1, 0, -1, 0, -1]],
[['passengers', 'felt', 'safe', 'due', 'to', 'visible', 'security'], [0, 0, 1, 0, 0, 0, 2], [0, 0, 1, 0, 0, 0, 1]],
[['comfort', 'is', 'decent', 'for', 'short', 'trips'], [1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]],
[['staff', 'provided', 'friendly', 'service', 'throughout'], [0, 0, 0, 1, 0], [0, 0, 0, 1, 0]],
[['facilities', 'are', 'outdated', 'in', 'some', 'stations'], [1, 0, 0, 0, 0, 0], [-1, 0, -1, 0, 0, 0]],
[['cleanliness', 'was', 'neutral', 'but', 'not', 'bad'], [1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]],
[['security', 'officers', 'helped', 'boost', 'safety', 'feeling'], [0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 1, 0]],
[['service', 'from', 'staff', 'was', 'helpful'], [1, 0, 0, 0, 0], [1, 0, 0, 0, 0]],
[['the', 'ride', 'was', 'comfortable', 'and', 'smooth'], [0, 0, 0, 1, 2, 0], [0, 0, 0, 1, 1, 0]],
[['comfort', 'could', 'be', 'improved'], [1, 0, 0, 0], [0, 0, 0, 0]],
[['safety', 'measures', 'seem', 'strict', 'and', 'working'], [1, 2, 0, 0, 0, 0], [1, 1, 0, 1, 0, 1]],
[['dirty', 'stations', 'show', 'lack', 'of', 'cleanliness'], [0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, -1]],
[['service', 'needs', 'major', 'upgrade'], [1, 0, 0, 0], [-1, 0, 0, -1]],
[['facilities', 'were', 'limited', 'but', 'usable'], [1, 0, 0, 0, 2], [0, 0, -1, 0, 0]],
[['cleanliness', 'has', 'always', 'been', 'great'], [1, 0, 0, 0, 2], [1, 0, 0, 0, 1]],
[['the', 'overall', 'comfort', 'was', 'mediocre'], [0, 0, 1, 0, 0], [0, 0, 0, 0, 0]],
[['poor', 'service', 'experience', 'this', 'time'], [0, 1, 2, 0, 0], [-1, -1, -1, 0, 0]],
[['safety', 'was', 'not', 'a', 'concern'], [1, 0, 0, 0, 0], [1, 0, 0, 0, 0]],
[['facilities', 'have', 'gotten', 'better', 'recently'], [1, 0, 0, 0, 0], [1, 0, 0, 1, 0]],
[['comfort', 'and', 'cleanliness', 'both', 'very', 'good'], [1, 0, 1, 0, 0, 0], [1, 0, 1, 0, 0, 1]],
[['service', 'was', 'slow', 'but', 'still', 'okay'], [1, 0, 0, 0, 0, 0], [-1, 0, -1, 0, 0, 0]],
[['safety', 'impressed', 'us', 'during', 'late', 'hours'], [1, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0]],
[['cleanliness', 'at', 'the', 'station', 'is', 'admirable'], [1, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 1]],
[['facilities', 'are', 'basic', 'but', 'sufficient'], [1, 0, 0, 0, 2], [0, 0, 0, 0, 0]],
[['comfort', 'in', 'the', 'train', 'was', 'awesome'], [1, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 1]],
[['security', 'and', 'safety', 'well', 'coordinated'], [0, 0, 1, 0, 0], [0, 0, 1, 0, 1]],
[['dirty', 'surfaces', 'show', 'lack', 'of', 'cleanliness'], [0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, -1]],
[['comfortable', 'seats', 'make', 'a', 'big', 'difference'], [1, 2, 0, 0, 0, 0], [1, 1, 0, 0, 0, 0]],
[['service', 'was', 'okay', 'this', 'morning'], [1, 0, 0, 0, 0], [0, 0, 0, 0, 0]],
[['safety', 'protocols', 'clearly', 'visible', 'to', 'all'], [1, 2, 0, 0, 0, 0], [1, 1, 0, 0, 0, 0]],
[['cleanliness', 'was', 'not', 'impressive'], [1, 0, 0, 0], [-1, 0, 0, -1]],
[['service', 'quality', 'needs', 'improvement'], [1, 2, 0, 0], [-1, -1, 0, -1]],
[['facilities', 'overall', 'were', 'fine'], [1, 0, 0, 0], [0, 0, 0, 0]],
[['comfort', 'levels', 'are', 'high'], [1, 2, 0, 0], [1, 1, 0, 1]],
[['good', 'cleanliness', 'adds', 'to', 'experience'], [0, 1, 0, 0, 0], [0, 1, 0, 0, 0]],
[['average', 'service', 'again', 'today'], [0, 1, 0, 0], [0, 0, 0, 0]],
[['I', 'felt', 'unsafe', 'at', 'night'], [0, 0, 1, 0, 0], [-1, 0, -1, 0, 0]],
[['cleanliness', 'standards', 'are', 'top-notch'], [1, 2, 0, 0], [1, 1, 0, 1]],
[['facilities', 'need', 'serious', 'upgrades'], [1, 0, 0, 0], [-1, 0, -1, -1]],
[['service', 'timing', 'is', 'bad'], [1, 2, 0, 0], [-1, -1, 0, -1]],
[['comfort', 'was', 'just', 'okay'], [1, 0, 0, 0], [0, 0, 0, 0]],
[['excellent', 'safety', 'track', 'record'], [0, 1, 0, 0], [0, 1, 0, 0]],
[['disappointed', 'with', 'cozy'], [0, 0, 1], [0, 0, -1]],
[['loved', 'the', 'equipment'], [0, 0, 1], [0, 0, 1]],
[['the', 'relaxing', 'is', 'terrible'], [0, 1, 2, 2], [0, -1, -1, -1]],
[['the', 'safety', 'is', 'okay'], [0, 1, 2, 2], [0, 0, 0, 0]],
[['the', 'pleasant', 'is', 'okay'], [0, 1, 2, 2], [0, 0, 0, 0]],
[['safety', 'could', 'be', 'better'], [1, 2, 2, 2], [-1, -1, -1, -1]],
[['very', 'poor', 'comfortable'], [0, 0, 1], [0, 0, -1]],
[['comfort', 'was', 'great'], [1, 2, 2], [1, 1, 1]],
[['very', 'good', 'hygiene'], [0, 0, 1], [0, 0, 1]],
[['help', 'was', 'average'], [1, 2, 2], [0, 0, 0]],
[['clean', 'environment'], [1, 2], [1, 1]],
[['neat', 'station'], [1, 2], [1, 1]],
[['tidy', 'but', 'crowded'], [1, 0, 0], [1, 0, 0]],
[['cleanliness', 'needs', 'work'], [1, 0, 0], [-1, 0, -1]],
[['safety', 'felt', 'okay'], [1, 0, 0], [0, 0, 0]],
[['comfortable', 'but', 'noisy'], [1, 0, 0], [1, 0, 0]],
[['good', 'facilities'], [0, 1], [0, 1]],
[['bad', 'service'], [0, 1], [0, -1]],
[['excellent', 'support'], [0, 1], [0, 1]],
[['helpful', 'staff'], [1, 2], [1, 1]],
[['facilities', 'are', 'sufficient'], [1, 0, 0], [0, 0, 0]],
[['dirty', 'toilets'], [1, 2], [-1, -1]],
[['love', 'the', 'service'], [0, 0, 1], [0, 0, 1]],
[['poor', 'assistance'], [0, 1], [0, -1]],
[['average', 'safety', 'measures'], [0, 1, 2], [0, 0, 0]],
[['awesome', 'service'], [0, 1], [0, 1]],
[['clean', 'and', 'tidy'], [1, 0, 2], [1, 0, 1]],
[['the', 'hygiene', 'was', 'terrible'], [0, 1, 0, 0], [-1, -1, 0, -1]],
[['cozy', 'interior'], [1, 2], [1, 1]],
[['safe', 'and', 'secure'], [1, 0, 2], [1, 0, 1]],
[['security', 'felt', 'weak'], [1, 0, 0], [-1, 0, -1]],
[['facilities', 'are', 'old'], [1, 0, 0], [-1, 0, -1]],
[['love', 'the', 'cleanliness'], [0, 0, 1], [0, 0, 1]],
[['tidy', 'environment'], [1, 2], [1, 1]],
[['awesome', 'facilities'], [0, 1], [0, 1]],
[['cleanliness', 'was', 'great'], [1, 0, 0], [1, 0, 1]],
[['average', 'comfort'], [0, 1], [0, 0]],
[['staff', 'were', 'rude'], [1, 0, 0], [-1, 0, -1]],
[['very', 'friendly', 'support'], [0, 0, 1], [0, 0, 1]],
[['relaxing', 'atmosphere'], [1, 2], [1, 1]],
[['service', 'improved'], [1, 0], [1, 1]],
[['good', 'safety', 'protocols'], [0, 1, 2], [0, 1, 1]],
[['security', 'was', 'bad'], [1, 0, 0], [-1, 0, -1]],
[['comfortable', 'but', 'hot'], [1, 0, 0], [0, 0, 0]],
[['cleanliness', 'is', 'average'], [1, 0, 0], [0, 0, 0]],
[['amazing', 'comfort'], [0, 1], [0, 1]],
[['dirty', 'floors'], [1, 2], [-1, -1]],
[['tidy', 'and', 'clean'], [1, 0, 2], [1, 0, 1]],
[['friendly', 'help'], [1, 2], [1, 1]],
[['slow', 'service'], [1, 2], [-1, -1]],
[['secure', 'surroundings'], [1, 2], [1, 1]],
[['great', 'staff'], [0, 1], [0, 1]],
[['cozy', 'seats'], [1, 2], [1, 1]],
[['safe', 'during', 'night'], [1, 0, 0], [1, 0, 0]],
[['not', 'very', 'clean'], [0, 0, 1], [0, 0, -1]],
[['poor', 'facilities'], [0, 1], [0, -1]],
[['okay', 'safety'], [0, 1], [0, 0]],
[['bad', 'staff'], [0, 1], [0, -1]],
[['friendly', 'and', 'helpful', 'staff'], [1, 0, 0, 2], [1, 0, 0, 1]],
[['support', 'is', 'excellent'], [1, 0, 0], [1, 0, 1]],
[['help', 'could', 'be', 'better'], [1, 0, 0, 0], [0, 0, 0, -1]],
[['cleanliness', 'improved', 'a', 'lot'], [1, 0, 0, 0], [1, 0, 0, 1]],
[['old', 'facilities'], [1, 2], [-1, -1]],
[['bad', 'comfort'], [0, 1], [0, -1]],
[['dirty', 'walls'], [1, 2], [-1, -1]],
[['awesome', 'support'], [0, 1], [0, 1]],
[['good', 'tidy', 'stations'], [0, 1, 2], [0, 1, 1]],
[['staff', 'were', 'great'], [1, 0, 0], [1, 0, 1]],
[['excellent', 'service'], [0, 1], [0, 1]],
[['very', 'poor', 'hygiene'], [0, 0, 1], [0, 0, -1]],
[['relaxing', 'train'], [1, 2], [1, 1]],
[['poor', 'cleanliness'], [0, 1], [0, -1]],
[['awesome', 'facilities'], [0, 1], [0, 1]],
[['average', 'service'], [0, 1], [0, 0]],
[['helpful', 'staff', 'again'], [1, 2, 0], [1, 1, 0]],
[['safe', 'and', 'calm'], [1, 0, 0], [1, 0, 0]],
[['security', 'was', 'fine'], [1, 0, 0], [0, 0, 0]],
[['hygiene', 'was', 'good'], [1, 0, 0], [1, 0, 1]],
[['facilities', 'need', 'upgrading'], [1, 0, 0], [-1, 0, -1]],
[['clean', 'but', 'old'], [1, 0, 0], [0, 0, -1]],
[['great', 'tidy', 'stations'], [0, 1, 2], [0, 1, 1]],
[['dirty', 'but', 'usable'], [1, 0, 0], [-1, 0, 0]],
[['okay', 'support'], [0, 1], [0, 0]],
[['bad', 'comfort'], [0, 1], [0, -1]],
[['neat', 'train'], [1, 2], [1, 1]],
[['cleanliness', 'was', 'bad'], [1, 0, 0], [-1, 0, -1]],
[['support', 'felt', 'slow'], [1, 0, 0], [-1, 0, -1]],
[['security', 'is', 'tight'], [1, 0, 0], [1, 0, 1]],
[['cozy', 'experience'], [1, 2], [1, 1]],
[['service', 'was', 'fair'], [1, 0, 0], [0, 0, 0]],
[['hygiene', 'is', 'poor'], [1, 0, 0], [-1, 0, -1]],
[['dirty', 'environment'], [1, 2], [-1, -1]],
[['clean', 'support'], [1, 2], [1, 1]],
[['staff', 'helped', 'a', 'lot'], [1, 0, 0, 0], [1, 0, 0, 1]],
[['neutral', 'about', 'the', 'hygiene'], [0, 0, 0, 1], [0, 0, 0, 0]],
[['clean', 'was', 'average'], [1, 2, 2], [0, 0, 0]],
[['very', 'good', 'cleanliness'], [0, 0, 1], [0, 0, 1]],
[['the', 'facilities', 'is', 'excellent'], [0, 1, 2, 2], [0, 1, 1, 1]],
[['very', 'good', 'pleasant'], [0, 0, 1], [0, 0, 1]],
[['the', 'danger', 'is', 'terrible'], [0, 1, 2, 2], [0, -1, -1, -1]],
[['the', 'neat', 'is', 'okay'], [0, 1, 2, 2], [0, 0, 0, 0]],
[['very', 'poor', 'assistance'], [0, 0, 1], [0, 0, -1]],
[['equipment', 'was', 'average'], [1, 2, 2], [0, 0, 0]],
[['cleanliness', 'could', 'be', 'better'], [1, 2, 2, 2], [-1, -1, -1, -1]],
[['amenities', 'was', 'great'], [1, 2, 2], [1, 1, 1]],
[['neutral', 'about', 'the', 'pleasant'], [0, 0, 0, 1], [0, 0, 0, 0]],
[['very', 'good', 'comfort'], [0, 0, 1], [0, 0, 1]],
[['services', 'was', 'average'], [1, 2, 2], [0, 0, 0]],
[['loved', 'the', 'safety'], [0, 0, 1], [0, 0, 1]],
[['safety', 'was', 'great'], [1, 2, 2], [1, 1, 1]],
[['loved', 'the', 'support'], [0, 0, 1], [0, 0, 1]],
[['the', 'cozy', 'is', 'terrible'], [0, 1, 2, 2], [0, -1, -1, -1]],
[['loved', 'the', 'amenities'], [0, 0, 1], [0, 0, 1]],
[['disappointed', 'with', 'safety'], [0, 0, 1], [0, 0, -1]],
[['neutral', 'about', 'the', 'tidy'], [0, 0, 0, 1], [0, 0, 0, 0]],
[['the', 'support', 'is', 'okay'], [0, 1, 2, 2], [0, 0, 0, 0]],
[['the', 'neat', 'is', 'terrible'], [0, 1, 2, 2], [0, -1, -1, -1]],
[['service', 'is', 'satisfactory'], [1, 2, 2], [0, 0, 0]],
[['assistance', 'could', 'be', 'better'], [1, 2, 2, 2], [-1, -1, -1, -1]],
[['very', 'good', 'protection'], [0, 0, 1], [0, 0, 1]],
[['the', 'comfort', 'is', 'okay'], [0, 1, 2, 2], [0, 0, 0, 0]],
[['neutral', 'about', 'the', 'relaxing'], [0, 0, 0, 1], [0, 0, 0, 0]],
[['disappointed', 'with', 'help'], [0, 0, 1], [0, 0, -1]],
[['protection', 'was', 'average'], [1, 2, 2], [0, 0, 0]],
[['hygiene', 'was', 'average'], [1, 2, 2], [0, 0, 0]],
[['very', 'poor', 'support'], [0, 0, 1], [0, 0, -1]],
[['disappointed', 'with', 'tidy'], [0, 0, 1], [0, 0, -1]],
[['comfortable', 'was', 'great'], [1, 2, 2], [1, 1, 1]],
[['tidy', 'was', 'great'], [1, 2, 2], [1, 1, 1]],
[['clean', 'could', 'be', 'better'], [1, 2, 2, 2], [-1, -1, -1, -1]],
[['neat', 'is', 'satisfactory'], [1, 2, 2], [0, 0, 0]],
[['the', 'cozy', 'is', 'okay'], [0, 1, 2, 2], [0, 0, 0, 0]],
[['the', 'equipment', 'is', 'excellent'], [0, 1, 2, 2], [0, 1, 1, 1]],
[['tidy', 'is', 'satisfactory'], [1, 2, 2], [0, 0, 0]],
[['very', 'poor', 'comfortable'], [0, 0, 1], [0, 0, -1]],
[['support', 'is', 'satisfactory'], [1, 2, 2], [0, 0, 0]],
[['the', 'cozy', 'is', 'excellent'], [0, 1, 2, 2], [0, 1, 1, 1]],
[['the', 'secure', 'is', 'excellent'], [0, 1, 2, 2], [0, 1, 1, 1]],
[['very', 'poor', 'hygiene'], [0, 0, 1], [0, 0, -1]],
[['very', 'poor', 'staff'], [0, 0, 1], [0, 0, -1]],
[['the', 'cleanliness', 'is', 'okay'], [0, 1, 2, 2], [0, 0, 0, 0]],
[['the', 'help', 'is', 'okay'], [0, 1, 2, 2], [0, 0, 0, 0]],
[['the', 'protection', 'is', 'okay'], [0, 1, 2, 2], [0, 0, 0, 0]],
[['comfort', 'is', 'satisfactory'], [1, 2, 2], [0, 0, 0]],
[['the', 'pleasant', 'is', 'terrible'], [0, 1, 2, 2], [0, -1, -1, -1]],
[['neutral', 'about', 'the', 'staff'], [0, 0, 0, 1], [0, 0, 0, 0]],
[['the', 'safe', 'is', 'terrible'], [0, 1, 2, 2], [0, -1, -1, -1]],
[['the', 'help', 'is', 'terrible'], [0, 1, 2, 2], [0, -1, -1, -1]],
[['pleasant', 'could', 'be', 'better'], [1, 2, 2, 2], [-1, -1, -1, -1]],
[['protection', 'was', 'great'], [1, 2, 2], [1, 1, 1]],
[['disappointed', 'with', 'comfort'], [0, 0, 1], [0, 0, -1]],
[['the', 'secure', 'is', 'terrible'], [0, 1, 2, 2], [0, -1, -1, -1]],
[['secure', 'was', 'great'], [1, 2, 2], [1, 1, 1]],
[['very', 'good', 'comfortable'], [0, 0, 1], [0, 0, 1]],
[['hygiene', 'was', 'average'], [1, 2, 2], [0, 0, 0]],
[['assistance', 'was', 'great'], [1, 2, 2], [1, 1, 1]],
[['comfortable', 'could', 'be', 'better'], [1, 2, 2, 2], [-1, -1, -1, -1]],
[['loved', 'the', 'assistance'], [0, 0, 1], [0, 0, 1]],
[['disappointed', 'with', 'tidy'], [0, 0, 1], [0, 0, -1]],
[['very', 'good', 'tidy'], [0, 0, 1], [0, 0, 1]],
[['staff', 'could', 'be', 'better'], [1, 2, 2, 2], [-1, -1, -1, -1]],
[['infrastructure', 'was', 'great'], [1, 2, 2], [1, 1, 1]],
[['disappointed', 'with', 'service'], [0, 0, 1], [0, 0, -1]],
[['very', 'poor', 'help'], [0, 0, 1], [0, 0, -1]],
[['very', 'poor', 'danger'], [0, 0, 1], [0, 0, -1]],
[['very', 'poor', 'pleasant'], [0, 0, 1], [0, 0, -1]],
[['the', 'support', 'is', 'terrible'], [0, 1, 2, 2], [0, -1, -1, -1]],
[['very', 'poor', 'cozy'], [0, 0, 1], [0, 0, -1]],
[['very', 'good', 'cozy'], [0, 0, 1], [0, 0, 1]],
[['loved', 'the', 'safe'], [0, 0, 1], [0, 0, 1]],
[['the', 'relaxing', 'is', 'okay'], [0, 1, 2, 2], [0, 0, 0, 0]],
[['the', 'infrastructure', 'is', 'okay'], [0, 1, 2, 2], [0, 0, 0, 0]],
[['loved', 'the', 'infrastructure'], [0, 0, 1], [0, 0, 1]],
[['cleanliness', 'is', 'satisfactory'], [1, 2, 2], [0, 0, 0]],
[['neutral', 'about', 'the', 'amenities'], [0, 0, 0, 1], [0, 0, 0, 0]],
[['average', 'experience', 'with', 'cleanliness'], [0, 0, 0, 1], [0, 0, 0, 0]],
[['secure', 'was', 'really', 'bad'], [1, 0, 0, 0], [-1, 0, 0, 0]],
[['danger', 'felt', 'very', 'nice'], [1, 0, 0, 0], [1, 0, 0, 0]],
[['help', 'felt', 'very', 'nice'], [1, 0, 0, 0], [1, 0, 0, 0]],
[['i', 'dislike', 'the', 'comfortable'], [0, 0, 0, 1], [0, 0, 0, -1]],
[['average', 'experience', 'with', 'support'], [0, 0, 0, 1], [0, 0, 0, 0]],
[['cleanliness', 'was', 'really', 'good'], [1, 0, 0, 0], [1, 0, 0, 0]],
[['i', 'loved', 'the', 'service'], [0, 0, 0, 1], [0, 0, 0, 1]],
[['equipment', 'was', 'just', 'okay'], [1, 0, 0, 0], [0, 0, 0, 0]],
[['cozy', 'felt', 'very', 'poor'], [1, 0, 0, 0], [-1, 0, 0, 0]],
[['the', 'MRT', 'makes', 'travel', 'easy'], [0, 0, 0, 0, 1], [0, 1, 1, 1, 1]],
[['delays', 'are', 'becoming', 'frequent'], [1, 0, 0, 0], [-1, -1, -1, -1]],
[['comfortable', 'seating', 'is', 'a', 'plus'], [1, 1, 0, 0, 0], [1, 1, 0, 0, 1]],
[['not', 'enough', 'handrails'], [0, 0, 1], [-1, -1, -1]],
[['fast', 'and', 'reliable', 'service'], [1, 0, 1, 1], [1, 0, 1, 1]],
[['stations', 'are', 'well', 'maintained'], [1, 0, 0, 1], [1, 0, 0, 1]],
[['poor', 'air', 'circulation'], [1, 1, 0], [-1, -1, 0]],
[['reliable', 'and', 'safe'], [1, 0, 1], [1, 0, 1]],
[['ticket', 'prices', 'are', 'affordable'], [1, 1, 0, 0], [1, 1, 0, 1]],
[['crowded', 'during', 'rush', 'hour'], [1, 0, 0, 0], [-1, -1, -1, -1]],
[['easy', 'to', 'use', 'system'], [1, 0, 0, 0], [1, 0, 0, 1]],
[['frequent', 'and', 'efficient', 'service'], [1, 0, 1, 1], [1, 0, 1, 1]],
[['poor', 'lighting', 'at', 'some', 'stations'], [1, 1, 0, 0, 0], [-1, -1, 0, 0, 0]],
[['good', 'connection', 'between', 'lines'], [1, 1, 0, 0], [1, 1, 0, 1]],
[['stations', 'lack', 'basic', 'amenities'], [1, 1, 1, 1], [-1, -1, -1, -1]],
[['staff', 'is', 'generally', 'polite'], [1, 0, 0, 1], [1, 0, 0, 1]],
[['late', 'night', 'trains', 'are', 'infrequent'], [1, 0, 1, 0, 0], [-1, 0, -1, 0, 0]],
[['clean', 'and', 'comfortable', 'travel'], [1, 0, 1, 1], [1, 0, 1, 1]],
[['accessibility', 'is', 'good', 'for', 'all'], [1, 0, 0, 0, 0], [1, 0, 0, 1, 1]],
[['the', 'MRT', 'is', 'very', 'convenient', 'and', 'fast'], [0, 0, 0, 0, 1, 0, 1], [1, 1, 0, 0, 1, 1, 1]],
[['delays', 'cause', 'significant', 'disruption'], [1, 0, 0, 0], [-1, -1, -1, -1]],
[['comfortable', 'and', 'smooth', 'journey'], [1, 0, 1, 1], [1, 0, 1, 1]],
[['not', 'enough', 'information', 'during', 'delays'], [0, 0, 0, 0, 1], [-1, -1, -1, -1, -1]],
[['fast', 'and', 'efficient', 'transport'], [1, 0, 1, 1], [1, 0, 1, 1]],
[['stations', 'are', 'well-lit', 'and', 'safe'], [1, 0, 0, 0, 1], [1, 0, 0, 1, 1]],
[['poor', 'sound', 'insulation'], [1, 1, 0], [-1, -1, 0]],
[['reliable', 'and', 'on', 'time'], [1, 0, 0, 0], [1, 0, 1, 1]],
[['ticket', 'prices', 'are', 'too', 'high'], [1, 1, 0, 0, 0], [-1, -1, 0, 0, -1]],
[['crowded', 'trains', 'are', 'uncomfortable'], [1, 1, 0, 1], [-1, -1, 0, -1]],
[['easy', 'and', 'quick', 'access'], [1, 0, 1, 1], [1, 0, 1, 1]],
[['frequent', 'and', 'reliable', 'service'], [1, 0, 1, 1], [1, 0, 1, 1]],
[['poor', 'maintenance', 'of', 'facilities'], [1, 1, 0, 1], [-1, -1, 0, -1]],
[['good', 'and', 'clear', 'signage'], [1, 0, 1, 1], [1, 0, 1, 1]],
[['stations', 'lack', 'seating'], [1, 1, 1], [-1, -1, -1]],
[['staff', 'is', 'helpful', 'and', 'friendly'], [1, 0, 1, 0, 1], [1, 0, 1, 1, 1]],
[['late', 'and', 'inconsistent', 'service'], [1, 0, 1, 1], [-1, 0, -1, -1]],
[['clean', 'and', 'modern', 'trains'], [1, 0, 1, 1], [1, 0, 1, 1]],
[['accessibility', 'is', 'poor', 'for', 'some'], [1, 0, 0, 0, 0], [-1, 0, -1, -1, -1]],
[['the', 'MRT', 'is', 'a', 'good', 'option'], [0, 0, 0, 0, 0, 0], [1, 1, 0, 0, 1, 1]],
[['delays', 'are', 'rare'], [1, 0, 0], [1, 1, 1]],
[['comfortable', 'and', 'quiet', 'commute'], [1, 0, 1, 1], [1, 0, 1, 1]],
[['not', 'enough', 'updates', 'during', 'disruptions'], [0, 0, 0, 0, 1], [-1, -1, -1, -1, -1]],
[['fast', 'and', 'convenient', 'travel'], [1, 0, 1, 1], [1, 0, 1, 1]],
[['stations', 'are', 'well-equipped', 'and', 'clean'], [1, 0, 0, 0, 1], [1, 0, 0, 1, 1]],
[['poor', 'mobile', 'reception'], [1, 1, 0], [-1, -1, 0]],
[['reliable', 'and', 'safe', 'transportation'], [1, 0, 1, 1], [1, 0, 1, 1]],
[['ticket', 'prices', 'are', 'value', 'for', 'money'], [1, 1, 0, 1, 0, 0], [1, 1, 0, 1, 1, 1]],
[['crowded', 'and', 'hot', 'during', 'peak', 'hours'], [1, 0, 1, 0, 0, 0], [-1, -1, -1, -1, -1, -1]],
[['easy', 'and', 'fast', 'access', 'to', 'city'], [1, 0, 1, 1, 0, 0], [1, 0, 1, 1, 1, 1]],
[['frequent', 'and', 'efficient', 'train', 'service'], [1, 0, 1, 1, 1], [1, 0, 1, 1, 1]],
[['fast', 'MRT', 'service'], [1, 1, 1], [1, 1, 1]],
[['clean', 'and', 'tidy', 'trains'], [1, 0, 1, 1], [1, 0, 1, 1]],
[['comfortable', 'seats', 'always'], [1, 1, 0], [1, 1, 0]],
[['safe', 'travel', 'at', 'night'], [1, 1, 0, 0], [1, 1, 0, 0]],
[['good', 'customer', 'service'], [1, 1, 0], [1, 1, 0]],
[['modern', 'MRT', 'stations'], [1, 1, 1], [1, 1, 1]],
[['easy', 'platform', 'access'], [1, 1, 1], [1, 1, 1]],
[['clean', 'and', 'efficient'], [1, 0, 1], [1, 0, 1]],
[['comfortable', 'ride', 'everyday'], [1, 1, 0], [1, 1, 0]],
[['safe', 'and', 'reliable'], [1, 1, 1], [1, 1, 1]],
[['cleanliness', 'is', 'great'], [1, 0, 1], [1, 0, 1]],
[['comfortable', 'journey'], [1, 1], [1, 1]],
[['safe', 'and', 'secure', 'travel'], [1, 1, 1, 1], [1, 1, 1, 1]],
[['excellent', 'service', 'always'], [1, 1, 0], [1, 1, 0]],
[['modern', 'and', 'clean'], [1, 0, 1], [1, 0, 1]],
[['easy', 'MRT', 'access'], [1, 1, 1], [1, 1, 1]],
[['clean', 'MRT', 'system'], [1, 1, 1], [1, 1, 1]],
[['dirty', 'and', 'smelly', 'seats'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['crowded', 'and', 'uncomfortable', 'ride'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['unsafe', 'and', 'dangerous', 'tracks'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['old', 'and', 'broken', 'escalators'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['rude', 'and', 'unhelpful', 'staff'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['filthy', 'and', 'stained', 'floors'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['cramped', 'and', 'stuffy', 'trains'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['hazardous', 'and', 'precarious', 'platforms'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['malfunctioning', 'and', 'inadequate', 'facilities'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['terrible', 'and', 'awful', 'service'], [1, 0, 1, 2], [-1, -1, -1, -1]],
[['comfortable', 'train', 'ride'], [1, 1, 1], [1, 1, 1]],
[['safe', 'MRT', 'journey'], [1, 1, 1], [1, 1, 1]],
[['efficient', 'MRT', 'service'], [1, 1, 1], [1, 1, 1]],
[['cleanliness', 'good'], [1, 1], [1, 1]],
[['comfortable', 'seats'], [1, 1], [1, 1]],
[['safe', 'traveling'], [1, 1], [1, 1]],
[['friendly', 'service'], [1, 1], [1, 1]],
[['modern', 'MRT'], [1, 1], [1, 1]],
[['easy', 'access'], [1, 1], [1, 1]],
[['clean', 'trains'], [1, 1], [1, 1]],
[['comfortable', 'ride'], [1, 1], [1, 1]],
[['safe', 'transport'], [1, 1], [1, 1]],
[['efficient', 'MRT'], [1, 1], [1, 1]],
[['cleanliness', 'maintained'], [1, 1], [1, 1]],
[['comfortable', 'traveling'], [1, 1], [1, 1]],
[['safe', 'and', 'sound'], [1, 1, 1], [1, 1, 1]],
[['excellent', 'staff'], [1, 1], [1, 1]],
[['modern', 'facilities'], [1, 1], [1, 1]],
[['easy', 'navigation'], [1, 1], [1, 1]],
[['clean', 'and', 'tidy'], [1, 0, 1], [1, 0, 1]],
[['comfortable', 'and', 'fast'], [1, 0, 1], [1, 0, 1]],
[['safe', 'and', 'convenient'], [1, 0, 1], [1, 0, 1]],
[['efficient', 'and', 'reliable'], [1, 0, 1], [1, 0, 1]],
[['cleanliness', 'improved'], [1, 1], [1, 1]],
[['comfortable', 'and', 'relaxing'], [1, 0, 1], [1, 0, 1]],
[['safe', 'and', 'secure'], [1, 0, 1], [1, 0, 1]],
[['great', 'customer', 'care'], [1, 1, 0], [1, 1, 0]],
[['modern', 'and', 'updated'], [1, 0, 1], [1, 0, 1]],
[['easy', 'and', 'fast'], [1, 0, 1], [1, 0, 1]],
[['clean', 'and', 'bright'], [1, 0, 1], [1, 0, 1]],
[['comfortable', 'and', 'smooth'], [1, 0, 1], [1, 0, 1]],
[['safe', 'and', 'efficient'], [1, 0, 1], [1, 0, 1]],
[['efficient', 'and', 'quick'], [1, 0, 1], [1, 0, 1]],
[['dirty', 'MRT', 'trains'], [1, 1, 1], [-1, -1, -1]],
[['uncomfortable', 'seats'], [1, 1], [-1, -1]],
[['unsafe', 'at', 'night'], [1, 0, 0], [-1, -1, 0]],
[['poor', 'customer', 'service'], [1, 1, 0], [-1, -1, 0]],
[['old', 'MRT', 'stations'], [1, 1, 1], [-1, -1, -1]],
[['difficult', 'access'], [1, 1], [-1, -1]],
[['dirty', 'and', 'smelly'], [1, 0, 0], [-1, 0, -1]],
[['uncomfortable', 'and', 'slow'], [1, 0, 0], [-1, 0, -1]],
[['unsafe', 'and', 'unreliable'], [1, 0, 0], [-1, 0, -1]],
[['inefficient', 'and', 'slow'], [1, 0, 0], [-1, 0, -1]],
[['lack', 'cleanliness'], [1, 1], [-1, -1]],
[['very', 'uncomfortable'], [1, 1], [-1, -1]],
[['felt', 'unsafe', 'always'], [1, 0, 0], [-1, -1, 0]],
[['terrible', 'service'], [1, 1], [-1, -1]],
[['old', 'and', 'dirty'], [1, 0, 0], [-1, 0, -1]],
[['poor', 'access', 'always'], [1, 1, 0], [-1, -1, 0]],
[['dirty', 'MRT', 'often'], [1, 1, 0], [-1, -1, 0]],
[['uncomfortable', 'and', 'rough'], [1, 0, 0], [-1, 0, -1]],
[['unsafe', 'and', 'late'], [1, 0, 0], [-1, 0, -1]],
[['slow', 'and', 'rude'], [1, 0, 0], [-1, 0, -1]],
[['extremely', 'dirty'], [1, 1], [-1, -1]],
[['highly', 'uncomfortable'], [1, 1], [-1, -1]],
[['felt', 'very', 'unsafe'], [1, 0, 0], [-1, -1, 0]],
[['worst', 'service', 'ever'], [1, 1, 0], [-1, -1, 0]],
[['dilapidated', 'and', 'filthy'], [1, 0, 0], [-1, 0, -1]],
[['inaccessible', 'stations'], [1, 1], [-1, -1]],
[['dirty', 'MRT', 'daily'], [1, 1, 0], [-1, -1, 0]],
[['uncomfortable', 'and', 'bumpy'], [1, 0, 0], [-1, 0, -1]],
[['unsafe', 'and', 'delayed'], [1, 0, 0], [-1, 0, -1]],
[['inefficient', 'and', 'unhelpful'], [1, 0, 0], [-1, 0, -1]],
[['cleanliness', 'okay'], [1, 1], [0, 0]],
[['seats', 'are', 'fine'], [1, 0, 0], [0, 0, 0]],
[['felt', 'quite', 'safe'], [1, 0, 0], [0, 0, 0]],
[['average', 'service'], [1, 1], [0, 0]],
[['facilities', 'adequate'], [1, 1], [0, 0]],
[['access', 'is', 'okay'], [1, 0, 0], [0, 0, 0]],
[['trains', 'clean', 'sometimes'], [1, 1, 0], [0, 0, 0]],
[['ride', 'is', 'normal'], [1, 0, 0], [0, 0, 0]],
[['safety', 'is', 'average'], [1, 0, 0], [0, 0, 0]],
[['service', 'okay'], [1, 1], [0, 0]],
[['cleanliness', 'average'], [1, 1], [0, 0]],
[['seats', 'moderate'], [1, 1], [0, 0]],
[['felt', 'generally', 'safe'], [1, 0, 0], [0, 0, 0]],
[['standard', 'service'], [1, 1], [0, 0]],
[['facilities', 'decent'], [1, 1], [0, 0]],
[['access', 'is', 'fair'], [1, 0, 0], [0, 0, 0]],
[['trains', 'clean', 'usual'], [1, 1, 0], [0, 0, 0]],
[['ride', 'reasonable'], [1, 1], [0, 0]],
[['safety', 'good'], [1, 1], [0, 0]],
[['service', 'fine'], [1, 1], [0, 0]]
]


output_csv_path = "try tokens.csv"  # You can change the filename if needed.

with open(output_csv_path, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['tokens', 'tags', 'polarities'])  # Write the header row

    for row in data:
        # Ensure that each row has exactly three elements to prevent errors.
        if len(row) == 3:
            tokens, tags, polarities = row
            writer.writerow([tokens, tags, polarities])
        else:
            print(f"Skipping row with invalid format: {row}") #handle rows that don't have enough columns.

print(f"Data successfully written to {output_csv_path}")

Data successfully written to try tokens.csv
